In [143]:
import pandas as pd, os
import numpy as np 
import copy
import math
from tqdm.notebook import tqdm 
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

# 讀取RawData

In [144]:
# 測量值紀錄表
#後來抓的 lab含'Anti-HBc', 'Anti-HBs', 'Anti-HCV'
lab_df3 = pd.read_csv('./dataset/drug_lab_base_order/Lab_1_utf8.txt', encoding='utf-8') 
lab_df4 = pd.read_csv('./dataset/drug_lab_base_order/Lab_2__utf8.txt', encoding='utf-8')

drug_df = pd.read_csv('./dataset/drug_lab_base_order/Drug2016_20_utf8.txt',encoding='utf-8', on_bad_lines='error') #用藥紀錄表

peri_df1 = pd.read_csv('./dataset/drug_lab_base_order/periHD2016_18_utf8.txt', encoding='utf-8')
peri_df2 = pd.read_csv('./dataset/drug_lab_base_order/periHD2019_20_utf8.txt', encoding='utf-8')

base_df = pd.read_csv('./dataset/drug_lab_base_order/Base2016_20_utf8.txt', encoding='utf-8')

order_df = pd.read_csv('./dataset/drug_lab_base_order/Order2016_20_utf8.txt', encoding='utf-8')
# 合併表格
lab_df_Anti = pd.concat([lab_df3,lab_df4])

# 資料格式整理
# lab_df = lab_df.sort_values(["ID","報告日"])#按照ID進行排序再將同一個ID按照報告日排序

drug_df.loc[:,'CHOP3CDATE'] = drug_df.loc[:,'CHOP3CDATE'].astype('int64')

drug_df.loc[:,'CHOP3CDATE'] = drug_df.loc[:,'CHOP3CDATE'] + 191100000000

drug_df.loc[:,'CHOP3CDATE'] = pd.to_datetime(drug_df.loc[:,'CHOP3CDATE'], format='%Y%m%d%H%M')
drug_df = drug_df.sort_values(["ID","CHOP3CDATE"])

# order 醫囑建議：
# 包含後續可能會用到的洗腎時長、透析液Ca(越低越可能掉血壓)
order_df.loc[:,'VCHCDATE'] = pd.to_datetime(order_df.loc[:,'VCHCDATE'], format='%Y/%m/%d %H:%M')

lab_df_Anti.loc[:,'報告日'] = pd.to_datetime(lab_df_Anti.loc[:,'報告日'], format='%Y/%m/%d %H:%M')
lab_df_Anti = lab_df_Anti.sort_values(["ID","報告日"])#按照ID進行排序再將同一個ID按照報告日排序

base_df['生日']= pd.to_datetime(base_df['生日'])
base_df['生日']= base_df['生日'].dt.date

In [145]:
li = drug_df['CHOP3DRGNO'].unique()
drug_df = drug_df[(drug_df['CHOP3DRGNO'] == 'IRECO2 ') | (drug_df['CHOP3DRGNO'] == 'IDARB0 ')]

In [146]:
#每個表格有交集的ID設成Ture沒有交集設成False
peri_df = pd.read_csv('./dataset/outpatient_clinic(1).csv')
lab_ID = list(set(lab_df_Anti['ID'].values))
peri_ID = list(set(peri_df['ID'].values))
drug_ID = list(set(drug_df['ID'].values))

patient_ID = list(set(lab_ID).intersection(peri_ID, drug_ID))
lab_df_Anti = lab_df_Anti[lab_df_Anti['ID'].isin(patient_ID)]

# peri_df = peri_df[peri_df['ID'].isin(patient_ID)]
drug_df = drug_df[drug_df['ID'].isin(patient_ID)]

In [147]:
peri_df.shape

(309556, 62)

In [148]:
print(len(set(lab_df_Anti['ID'])))
print(len(set(peri_df['ID'])))
print(len(set(drug_df['ID'])))

1051
1051
1051


In [149]:
lab_df_Anti.shape

(1043829, 5)

In [150]:
# 資料格式整理
lab_df_Anti.loc[:,'報告日'] = pd.to_datetime(lab_df_Anti.loc[:,'報告日'])
lab_df_Anti = lab_df_Anti.sort_values(["ID","報告日"])#按照ID進行排序再將同一個ID按照報告日排序

peri_df.loc[:,'洗腎紀錄時間'] = pd.to_datetime(peri_df.loc[:,'洗腎紀錄時間'])
peri_df['透析開始時間']= pd.to_datetime(peri_df['透析開始時間'])
peri_df['透析開始時間']= peri_df['透析開始時間'].dt.date
peri_df = peri_df.sort_values(["ID","洗腎紀錄時間"])
print(peri_df.shape)

(309556, 62)


In [157]:
#粗布估計miss狀況

In [162]:
count_miss = ['開始血壓SBP', '開始血壓DBP', '開始脈搏', '體重1開始', '體重2標準體重', '體重結束', '體重實際脫水', '體重機器顯示', '體溫', 'FISTULA', 'GRAFT', 'DLC', 'PERMCATH', ]
for i in count_miss:
    print(i, " - ",peri_df[i].isnull().sum()/peri_df.shape[0]*100)

開始血壓SBP  -  0.6535166496530515
開始血壓DBP  -  0.6580392562250449
開始脈搏  -  1.306710255979532
體重1開始  -  0.6454405664887775
體重2標準體重  -  0.3110907234878342
體重結束  -  1.2714985333832973
體重實際脫水  -  1.3112328625515253
體重機器顯示  -  1.186215095168564
體溫  -  0.9171200041349546
FISTULA  -  17.296062747935753
GRAFT  -  87.56380105699778
DLC  -  99.91600873509155
PERMCATH  -  93.60115778728243


In [158]:
peri_df.columns

Index(['index', 'ID', '洗腎紀錄時間', '急診門診住院', '護理交班事項', '透析開始時間', '開始血壓SBP',
       '開始血壓DBP', '開始脈搏', '體重1開始', '體重2標準體重', '體重3應脫水', '體重其他', '體溫',
       'FISTULA', 'GRAFT', 'DLC', 'PERMCATH', 'EXITSITE', 'DLC順暢度', '血流強度',
       '血管彈性', '瘀青', '感染徵兆', '穿刺', '血管通路理學篩檢其他', '跌倒_年齡', '跌倒_跌倒病史', '跌倒_意識狀態',
       '跌倒_下肢肌力測驗', '跌倒_如廁能力', '跌倒_活動狀態', '跌倒_藥物使用', '入院方式', '意識狀態', '呼吸系統',
       '呼吸系統5其他', '心臟血管系統', '心臟血管系統其他', '透析時間結束', '結束血壓SBP', '結束血壓DBP', '結束脈搏',
       '體重結束', '體重實際脫水', '體重機器顯示', '血管廔管', '血管廔管2', '血管廔管3',
       'Hollow fiber clot', 'V-chamber clot', 'A-chamber clot', '處置其他結束',
       '透析後評估項目1', '透析後評估項目2', '透析後評估項目3', '透析後評估項目4', '透析後評估項目5', '透析後評估項目6',
       'DIAG585', 'DIAG585\nSTATDATE', 'DIAG585\nENDDATE'],
      dtype='object')

# 建構基本欄位(data_df)

In [163]:
#創新的
data_df = pd.DataFrame()
# T0 Featrue
data_df["index"], data_df["ID"], data_df["洗腎紀錄時間"], data_df["洗腎紀錄時間去時分"],data_df["急診門診住院"],data_df['入院方式'],data_df["FISTULA"], data_df["GRAFT"], data_df["DLC"],data_df["PERMCATH"],data_df["處置其他結束"]= \
peri_df["index"], peri_df["ID"], peri_df["洗腎紀錄時間"], peri_df["透析開始時間"],peri_df["急診門診住院"],peri_df['入院方式'],peri_df["FISTULA"], peri_df["GRAFT"], peri_df["DLC"],peri_df["PERMCATH"],peri_df["處置其他結束"]
data_df["開始血壓SBP"] = peri_df["開始血壓SBP"]
data_df["開始血壓DBP"] = peri_df["開始血壓DBP"]
data_df["開始脈搏"] = peri_df["開始脈搏"]
# T1 Featrue
# T2 Featrue
data_df["體溫"] = peri_df["體溫"]
# T3 Featrue
data_df["結束脈搏"] = peri_df["結束脈搏"]
data_df["Hollow fiber clot"] = peri_df["Hollow fiber clot"]

In [164]:
# np.unique(data_df['Hollow fiber clot'].astype(str).values, return_counts=True)

In [165]:
data_df.index = range(len(data_df))
data_df = data_df.drop_duplicates(keep='first', inplace=False)
data_df['洗腎紀錄時間去時分']= pd.to_datetime(data_df['洗腎紀錄時間去時分'])
data_df['洗腎紀錄時間去時分']= data_df['洗腎紀錄時間去時分'].dt.date
data_df.reset_index(inplace=True, drop=True)

### 處理廔管

In [166]:
#廔管不是na補1 ,na補0
# data_df['FISTULA'].replace(["(Lt,Rt,)", '(Lt,)','(Rt,)','Fistula','Fistula(Lt,)','Fistula(Lt,Rt,)','Fistula(Rt,)'],[1,1,1,1,1,1,1], inplace=True) 
# data_df['GRAFT'].replace(['(Lt,)','(Lt,Rt,)','(Rt,)','Graft','Graft(Lt,)','Graft(Lt,Rt,)','Graft(Rt,)'],[1,1,1,1,1,1,1], inplace=True) 
# 2024/04/23修正新資料條件
data_df['FISTULA'].replace(["(Lt,Rt,)", '(Lt,)','(Rt,)','Fistula','Fistula(Lt,)','Fistula(Lt,Rt,)','Fistula(Rt,)','Lt', 'Rt', 'LtRt'],[1,1,1,1,1,1,1,1,1,1], inplace=True) 
data_df['GRAFT'].replace(['(Lt,)','(Lt,Rt,)','(Rt,)','Graft','Graft(Lt,)','Graft(Lt,Rt,)','Graft(Rt,)', 'Lt', 'Rt'],[1,1,1,1,1,1,1,1,1], inplace=True)

data_df['DLC'].replace(['(Lt,)', '(Rt,)', 'Femoral', 'Femoral(Lt,)', 'Femoral(Rt,)', 'Jugular(Rt,)', 'Jugular'],[1,1,1,1,1,1,1], inplace=True) 
data_df['PERMCATH'].replace(['Perm-cath'],[1], inplace=True) 
data_df["FISTULA"] = data_df["FISTULA"].fillna(0)
data_df["GRAFT"] = data_df["GRAFT"].fillna(0)
data_df["DLC"] = data_df["DLC"].fillna(0)
data_df["PERMCATH"] = data_df["PERMCATH"].fillna(0)

In [167]:
indexNames = data_df[(data_df['FISTULA']==0)&(data_df['GRAFT']==0)&(data_df['DLC']==0)&(data_df['PERMCATH']==0)].index
for i in indexNames:
    data_df['FISTULA'][i] = np.nan
    data_df['GRAFT'][i] = np.nan
    data_df['DLC'][i] = np.nan
    data_df['PERMCATH'][i] = np.nan


In [168]:
# 體重部分需要做額外的確認補植
groupby_list = ['ID','FISTULA','GRAFT','DLC','PERMCATH']
data_df.reset_index(inplace=True, drop=True)

temp_df = data_df[groupby_list]
print(print(data_df.isnull().sum()))
temp_df = temp_df.groupby(["ID"], as_index=False).apply(lambda group: group.bfill())
temp_df = temp_df.groupby(["ID"], as_index=False).apply(lambda group: group.ffill())
temp_df.reset_index(inplace=True, drop=True)
data_df[groupby_list] = temp_df
print(data_df.isnull().sum())

index                    0
ID                       0
洗腎紀錄時間                   0
洗腎紀錄時間去時分                0
急診門診住院                   0
入院方式                  2363
FISTULA               6663
GRAFT                 6663
DLC                   6663
PERMCATH              6663
處置其他結束                3815
開始血壓SBP               2023
開始血壓DBP               2037
開始脈搏                  4045
體溫                    2839
結束脈搏                  4216
Hollow fiber clot    68804
dtype: int64
None
index                    0
ID                       0
洗腎紀錄時間                   0
洗腎紀錄時間去時分                0
急診門診住院                   0
入院方式                  2363
FISTULA                764
GRAFT                  764
DLC                    764
PERMCATH               764
處置其他結束                3815
開始血壓SBP               2023
開始血壓DBP               2037
開始脈搏                  4045
體溫                    2839
結束脈搏                  4216
Hollow fiber clot    68804
dtype: int64


In [169]:
indexNames = data_df[(data_df['FISTULA'].isna())&(data_df['GRAFT'].isna())&(data_df['DLC'].isna())&(data_df['PERMCATH'].isna())].index
for i in indexNames:
    data_df['FISTULA'][i] = 1 # 
    data_df['GRAFT'][i] = 0
    data_df['DLC'][i] = 0
    data_df['PERMCATH'][i] = 0

In [170]:
data_df = data_df.reset_index(inplace=False, drop=True)
data_df['Catheter'] = data_df['DLC'] + data_df['PERMCATH']
for i in range(len(data_df)):
    if data_df['Catheter'][i]== 2:
        data_df['Catheter'][i]= 1

# 生理數值

In [171]:
# 需要抓天數
# 'ALBUMIN', 'ALT (SGPT)', 'Alk.phosphatase','BUN', 'Ca', 'Na', 'P','MCV','MCH','MCHC',  'HGB','K','AST (SGOT)','Glucose AC','Glucose PC','HGB','HCT','RBC','WBC','Platelet','Creatinine',= 90內的值，否則算miss或9999
# 'Ferritin', 'IRON/TIBC','Intact PTH', 'Iron','BUN (洗腎後)', 'LDL Cholesterol','Bilirubin-T','Cholesterol-T','CRP' = 180內的值，否則算miss或9999
# 'HBs Ag','Anti-HBc', 'Anti-HBs', 'Anti-HCV' = 以最近一筆，否則算miss或9999

# 90 days
lab_df_Anti['檢驗名稱'].replace(
                              ['K', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'MCV', 'MCHC', 'MCH', 'Na', 'Ca', 'BUN', 'AST (SGOT)', 'ALT (SGPT)', 'Alk.phosphatase']
                             ,['K', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'MCV', 'MCHC', 'MCH', 'Na', 'Ca', 'BUN', 'AST (SGOT)', 'ALT (SGPT)', 'Alk.phosphatase']
                             , inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['Glucose AC', 'Glucose,AC', ],'Glucose AC', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['Glucose,PC', 'Glucose PC', 'GLUCOSE PC', ],'Glucose PC', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['Albumin', 'ALBUMIN', ],'ALBUMIN', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['P  (Phosphorus)','P'],'P', inplace=True)

# 180 days
lab_df_Anti['檢驗名稱'].replace(
                              ['TIBC', 'Ferritin', 'BUN (洗腎後)', 'IRON/TIBC', ]
                             ,['TIBC', 'Ferritin', 'BUN (洗腎後)', 'IRON/TIBC', ]
                             , inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['INTACT PTH', 'Intact PTH', ],'Intact PTH', inplace=True)
lab_df_Anti['檢驗名稱'].replace(['IRON (Fe)', 'Iron', ],'Iron', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['LDL Cholesterol', 'LDL CHOLESTEROL', ],'LDL Cholesterol', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['Bilirubin-T.', 'BILI-T.', ],'Bilirubin-T', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['Cholesterol T', 'CHOLESTEROL T.', ],'Cholesterol-T', inplace=True) 
lab_df_Anti['檢驗名稱'].replace(['High Sensitivity CRP', 'CRP', ],'CRP', inplace=True) 

# 一次性判斷
lab_df_Anti['檢驗名稱'].replace(
                              ['Anti-HBs', 'HBs Ag', 'Anti-HBc', 'Anti-HCV', 'HbA1c', ]
                             ,['Anti-HBs', 'HBs Ag', 'Anti-HBc', 'Anti-HCV', 'HbA1c', ]
                             , inplace=True) 

# 暫時無用特徵
if True:
    lab_df_Anti['檢驗名稱'].replace(['Mg', 'Mg (Magnesium)', ],'Useless-Mg', inplace=True) 
    lab_df_Anti['檢驗名稱'].replace(['HDL Cholesterol', 'HDL CHOLESTEROL', ],'Useless-HDL Cholesterol', inplace=True) 
    lab_df_Anti['檢驗名稱'].replace(['Bilirubin-D', 'BILIRUBIN-D.', ],'Useless-Bilirubin-D', inplace=True) 
    lab_df_Anti['檢驗名稱'].replace(
                                ['RPR','VBG-HCO3-','ABG-HCO3-','LDH','r-GT','HBe Ag','Cl']
                                ,['Useless-RPR','Useless-VBG-HCO3','Useless-ABG-HCO3','Useless-LDH','Useless-r-GT','Useless-HBe-Ag','Useless-Cl']
                                , inplace=True) 

In [172]:
print('90天判定特徵')
# '、'.join
print((sorted(['K', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'MCV', 'MCHC', 'MCH', 'Na', 'Ca', 'BUN', 'AST (SGOT)', 'ALT (SGPT)', 'Alk.phosphatase','Glucose AC', 'Glucose PC', 'Albumin', 'P'])))
print('180天判定特徵')
print((sorted(['TIBC', 'Ferritin', 'BUN (洗腎後)', 'IRON/TIBC', 'Intact PTH', 'Iron', 'LDL Cholesterol', 'Bilirubin-T', 'Cholesterol-T', 'CRP'])))
# URR%: 180天 # CRP: 一年做一次  
print('一次性判定特徵')
print((sorted(['Anti-HBs', 'HBs Ag', 'Anti-HBc', 'Anti-HCV', 'HbA1c'])))

90天判定特徵
['ALT (SGPT)', 'AST (SGOT)', 'Albumin', 'Alk.phosphatase', 'BUN', 'Ca', 'Creatinine', 'Glucose AC', 'Glucose PC', 'HCT', 'HGB', 'K', 'MCH', 'MCHC', 'MCV', 'Na', 'P', 'Platelet', 'RBC', 'WBC']
180天判定特徵
['BUN (洗腎後)', 'Bilirubin-T', 'CRP', 'Cholesterol-T', 'Ferritin', 'IRON/TIBC', 'Intact PTH', 'Iron', 'LDL Cholesterol', 'TIBC']
一次性判定特徵
['Anti-HBc', 'Anti-HBs', 'Anti-HCV', 'HBs Ag', 'HbA1c']


### 從Lab DataFrame串BUN

In [173]:
# URR% - 需要以同一天的透析時間做計算!!!
# URR%有效區間為180天
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# new_BUN_SE_df = pd.DataFrame()
BUN_S = lab_df_Anti[lab_df_Anti['檢驗名稱']=='BUN']
BUN_S.loc[:,'報告日'] = pd.to_datetime(BUN_S.loc[:,'報告日']).dt.date
BUN_S.rename(columns={'DATANUM':'DATANUM(前)',},inplace = True)
BUN_E = lab_df_Anti[lab_df_Anti['檢驗名稱']=='BUN (洗腎後)']
BUN_E.loc[:,'報告日'] = pd.to_datetime(BUN_E.loc[:,'報告日']).dt.date
BUN_E.rename(columns={'DATANUM':'DATANUM(後)',},inplace = True)
BUN_SE = BUN_S.merge(BUN_E, on=['ID','報告日'], how='left')
print("洗腎前BUN筆數：{}  洗腎後BUN筆數：{}".format(BUN_S.shape[0], BUN_E.shape[0]))
BUN_SE['DATANUM(前)'] = BUN_SE['DATANUM(前)'].str.replace('Rechecked','')
BUN_SE['DATANUM(前)'] = pd.to_numeric(BUN_SE['DATANUM(前)'])
BUN_SE['DATANUM(後)'] = BUN_SE['DATANUM(後)'].str.replace('Rechecked','')
BUN_SE['DATANUM(後)'] = pd.to_numeric(BUN_SE['DATANUM(後)'])
BUN_SE['URR%'] = round((BUN_SE['DATANUM(前)']-BUN_SE['DATANUM(後)'])/BUN_SE['DATANUM(前)'] * 100,1)
BUN_SE['URR%'] = BUN_SE['URR%'].replace(np.nan, 0)

for i in range(len(BUN_SE)):
    if BUN_SE['URR%'][i]<0:
        # BUN_SE['URR%'][i] = round((BUN_SE['DATANUM(後)'][i]-BUN_SE['DATANUM(前)'][i])/BUN_SE['DATANUM(後)'][i] * 100,1)
        BUN_SE['URR%'][i] = 19999.0

BUN_SE['檢驗名稱']='URR%'
BUN_SE.rename(columns={'URR%':'DATANUM','DATAUNIT_y':'DATAUNIT'},inplace = True)
BUN_SE = BUN_SE[BUN_SE['DATANUM']!=0][['ID','報告日','檢驗名稱','DATANUM','DATAUNIT']]
print(f"能計算URR%的筆數：{BUN_SE.shape[0]}")
# 將URR%合併至Anti-lab-df
BUN_SE.loc[:,'報告日'] = pd.to_datetime(BUN_SE.loc[:,'報告日'])
lab_df_Anti = pd.concat([lab_df_Anti,BUN_SE])
print(lab_df_Anti.shape)

洗腎前BUN筆數：40952  洗腎後BUN筆數：8717
能計算URR%的筆數：6038
(1049867, 5)


In [174]:

# 抓取病人特徵 (lab_df_Anti)
# 04-19: add URR%
feature = ['K', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'MCV', 'MCHC', 'MCH', 'Na', 'K', 'Ca', 'BUN', 'AST (SGOT)', 'ALT (SGPT)', 'Alk.phosphatase', 'Glucose AC', 'Glucose PC', 'ALBUMIN', 'P',
           'TIBC', 'Ferritin', 'BUN (洗腎後)', 'IRON/TIBC', 'Intact PTH', 'Iron', 'LDL Cholesterol', 'Bilirubin-T', 'Cholesterol-T', 'CRP', 'URR%',
           'Anti-HBs', 'HBs Ag', 'Anti-HBc', 'Anti-HCV'
           ]
# **HbA1c後續糖尿病再處理**#
# 將每一個feature加進data的標頭裡然後項目裡都設成NaN
for i in feature:
    data_df[i] = np.nan
data_df.loc[:,'洗腎紀錄時間'] = pd.to_datetime(data_df.loc[:,'洗腎紀錄時間'])

### 根據檢驗值合理天數區間進行串聯

In [189]:
# 將病人的測量值抓入表中，因為每個測量值的時間點不太一定，若以洗腎紀錄時間去抓會有很多missing value，所以用距離洗腎紀錄時間最近的值來補值
if "第一階段處理結果-前半部merge檢驗資料.csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-前半部merge檢驗資料.csv")
else:
    for i in tqdm(set(data_df['ID'])): 
        patient_df = data_df[data_df['ID'] == i]  #在data裡面的ID
        feature_df = lab_df_Anti[lab_df_Anti['ID'] == i]    #在原先LAB檔案裡面的ID 等同patient_df的ID
        #在原先LAB檔案裡面的同一個ID裡面的 各個feature
        for j in feature:
            if j in ['ALT (SGPT)', 'AST (SGOT)', 'Albumin', 'Alk.phosphatase', 'BUN', 'Ca', 'Creatinine', 'Glucose AC', 'Glucose PC', 
                        'HCT', 'HGB', 'K', 'MCH', 'MCHC', 'MCV', 'Na', 'P', 'Platelet', 'RBC', 'WBC']: #90內的值，否則算miss或9999
                ths = 90
            elif j in ['BUN (洗腎後)', 'Bilirubin-T', 'CRP', 'Cholesterol-T', 'Ferritin', 
                        'IRON/TIBC', 'Intact PTH', 'Iron', 'LDL Cholesterol', 'TIBC','URR%']: #180內的值，否則算miss或9999
                ths = 180
            else:
                # ['HBs Ag','Anti-HBc', 'Anti-HBs', 'Anti-HCV', {其他特徵}]:#以最近一筆，否則算miss或9999
                ths = 99999999 #無限制
            one_feature_df = feature_df[feature_df['檢驗名稱'] == j]
            time1 = patient_df['洗腎紀錄時間'].values #data裡面我們設立的開始時間欄位
            time2 = one_feature_df['報告日'].values #原先LAB裡面報告日欄位
            if len(time2) == 0:
                continue
            else:    
                closestTimestamp = []
                for k in time1:
                    closestTimestamp.append(min(time2, key=lambda b: abs(k-b))) 
                count = 0
                for index, row in patient_df.iterrows():
                    if (data_df.loc[index, '洗腎紀錄時間'] - one_feature_df[one_feature_df['報告日'] == closestTimestamp[count]]['報告日']).values[0].days>ths:
                        # data_df.loc[index,j] = np.nan
                        data_df.loc[index,j] = "enough far"
                        # print(f"{} - enough far")
                        # print(f"{data_df.loc[index, '洗腎紀錄時間']} => {closestTimestamp[count]}")
                    else:
                        data_df.loc[index,j] = one_feature_df[one_feature_df['報告日'] == closestTimestamp[count]]['DATANUM'].values[0]
                    # data_df.loc[index,"抽血檢驗時間"] = closestTimestamp[count]
                    count+=1
    print(data_df['ID'].nunique(dropna=True))
    data_df.to_csv("./dataset/第一階段處理結果-前半部merge檢驗資料.csv", index=False, encoding='utf-8-sig')

In [190]:
data_df.replace("enough far", -9999, inplace=True)
data_df['URR%'].replace(9999, 19999, inplace=True)

In [192]:
temp = data_df[data_df.columns.drop(['處置其他結束','開始脈搏','結束脈搏','開始脈搏','開始血壓SBP', '開始血壓DBP','體溫','Hollow fiber clot'])]
temp = temp.isnull().sum(axis=1)
# temp = pd.DataFrame({"ID":data_df['ID'].values, "count":temp.values})
# temp = temp[temp['count']!=0]
# print(f"共 {temp.drop_duplicates(subset='ID').shape[0]} 個病人有遺失值")
# 原先17個特徵有320個病人有遺失值

In [193]:
temp

0         0
1         0
2         0
3         0
4         0
         ..
309551    2
309552    7
309553    7
309554    7
309555    7
Length: 309556, dtype: int64

In [23]:
# 透析液 Ca
# 串order 以最近一筆醫囑為主
# nan設為9999
order_df_filter = order_df[['ID','VCHCDATE','透析液 Ca']]
order_df_filter = order_df_filter.sort_values(["ID","VCHCDATE"])#按照ID進行排序再將同一個ID按照報告日排序
order_df_filter.loc[:,'VCHCDATE'] = pd.to_datetime(order_df_filter.loc[:,'VCHCDATE'])
data_df.loc[:,'洗腎紀錄時間'] = pd.to_datetime(data_df.loc[:,'洗腎紀錄時間'])
order_df_filter.head(5)

,ID,VCHCDATE,透析液 Ca
48300,000099H08,2020-05-16 16:47:00,3.5
47735,000099H08,2020-05-20 07:09:00,3.5
50303,000099H08,2020-05-21 08:49:00,3.5
48370,000099H08,2020-05-21 09:08:00,3.5
47247,000099H08,2020-05-26 09:05:00,3.5


### 從Order DataFrame串Ca

In [24]:
data_df['透析液 Ca'] = np.nan
for i in tqdm(set(data_df['ID'])): 
    patient_df = data_df[data_df['ID'] == i]  #在data裡面的ID
    feature_df = order_df_filter[order_df_filter['ID'] == i]    #在原先LAB檔案裡面的ID 等同patient_df的ID
    #在原先LAB檔案裡面的同一個ID裡面的 各個feature
    # j = Ca
    j = '透析液 Ca'
    time1 = patient_df['洗腎紀錄時間'].values #data裡面我們設立的開始時間欄位
    time2 = feature_df['VCHCDATE'].values #原先LAB裡面報告日欄位
    # break
    if len(time2) == 0:
        continue
    else:    
        closestTimestamp = []
        for k in time1:
            closestTimestamp.append(min(time2, key=lambda b: abs(k-b))) 
        count = 0
        for index, row in patient_df.iterrows():
            data_df.loc[index,j] = feature_df[feature_df['VCHCDATE'] == closestTimestamp[count]]['透析液 Ca'].values[0]
            # data_df.loc[index,"抽血檢驗時間"] = closestTimestamp[count]
            count+=1
    # break

  0%|          | 0/1051 [00:00<?, ?it/s]

In [25]:
# 處理入院方式 => 'nan', '助行器', '推床', '步行', '輪椅'
str_cat = {'步行':0,'助行器':1, '輪椅':2, '推床':3, 'nan':-9999}
data_df['入院方式'] = data_df['入院方式'].astype(str).apply(lambda x: str_cat[x])

### 處理BC肝

In [26]:
data_df['洗腎紀錄時間']= pd.to_datetime(data_df['洗腎紀錄時間'])
data_df.rename(columns={'Anti-HBc':'HBc','Anti-HBs':'HBs','Anti-HCV':'HCV','HBs Ag':'HBsAg'},inplace = True)
for i in tqdm(range(len(data_df['ID']))):
    if pd.isnull(data_df.HBsAg[i]) == False:
        if 'Non' in data_df['HBsAg'][i]:
            data_df['HBsAg'][i] = '0'
        elif 'Non' not in data_df['HBsAg'][i]:
            data_df['HBsAg'][i] = '1'
    
    if pd.isnull(data_df.HBc[i]) == False:
        if 'Non' in data_df['HBc'][i]:
            data_df['HBc'][i] = '0'
        elif 'Non' not in data_df['HBc'][i]:
            data_df['HBc'][i] = '1'
    
    if pd.isnull(data_df.HBs[i]) == False:
        if 'Non' in data_df['HBs'][i]:
            data_df['HBs'][i] = '0'
        elif 'Negative' in data_df['HBs'][i]:
            data_df['HBs'][i] = '0'
        elif 'Non' not in data_df['HBs'][i]:
            data_df['HBs'][i] = '1'
            
    if pd.isnull(data_df.HCV[i]) == False:
        if 'Non' in data_df['HCV'][i]:
            data_df['HCV'][i] = '0'
        elif 'Non' not in data_df['HCV'][i]:
            data_df['HCV'][i] = '1'


  0%|          | 0/309556 [00:00<?, ?it/s]

In [27]:
data_df.describe()

,index,洗腎紀錄時間,入院方式,FISTULA,GRAFT,DLC,PERMCATH,Catheter,RBC,MCH,Glucose PC,URR%,透析液 Ca
count,309556.000000,309556,309556.000000,309556.000000,309556.000000,309556.000000,309556.000000,309556.000000,309512.000000,309512.000000,47103.000000,295781.000000,308767.000000
mean,179308.629007,2018-08-29 19:27:08.859205632,-75.957387,0.840026,0.130632,0.000853,0.066660,0.066812,3.866656,29.070088,222.809800,75.997336,2.921494
min,4.000000,2016-01-01 07:35:00,-9999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.330000,17.900000,34.000000,7.500000,2.500000
25%,90895.750000,2017-07-03 17:39:15,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.450000,27.800000,143.000000,69.000000,2.500000
50%,180416.500000,2018-09-18 07:48:00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.810000,29.500000,193.000000,72.500000,3.000000
75%,268817.250000,2019-11-09 17:25:00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.210000,31.000000,287.000000,76.000000,3.500000
max,358822.000000,2020-12-31 18:45:00,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.460000,45.300000,527.000000,19999.000000,3.500000
std,103597.829404,NaN,870.305905,0.366583,0.336998,0.029191,0.249433,0.249696,0.646255,2.931175,104.218219,274.221324,0.423753


In [28]:
# # 修正新特徵
# 'Hollow fiber clot'後續再處理
new_feature = ['K', 'AST (SGOT)', 'Glucose AC', 'Glucose PC', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'LDL Cholesterol', 'Bilirubin-T', 'Cholesterol-T', 'TIBC', 'CRP']
data_df[new_feature] = data_df[new_feature].astype(str)
for f in new_feature:
    data_df[f] = data_df[f].apply(lambda x: re.sub(r'[^0-9\.-]', '', x))
    data_df[f] = data_df[f].str.replace("Rechecked", '')
    data_df[f].replace(['nan'],-9999.0,inplace=True)
    data_df[f] = pd.to_numeric(data_df[f])
    data_df[f].replace(np.nan,-9999.0,inplace=True)
# for f in new_feature:
#     print(f)
#     print(np.sort(data_df[f].unique()).astype(str))
#     # data_df[f] = pd.to_numeric(data_df[f])
# print("end")
    

In [29]:
#判斷HBV
# HBV後續在處理
data_df[['HBs','HBc','HBsAg']] = data_df[['HBs','HBc','HBsAg']].astype(float)

data_df['HBV'] = np.nan
for i in range(len(data_df['HBV'])):
    if (data_df['HBV'][i] == 1) |(data_df['HBV'][i] == 0):
        continue
    if data_df['HBs'][i] == 1:
        data_df['HBV'][i] = 0
    elif data_df['HBs'][i] == 0:
        if data_df['HBc'][i] == 1:
            data_df['HBV'][i] = 1
        elif data_df['HBc'][i] == 0:
            data_df['HBV'][i] = 0
        elif data_df['HBsAg'][i] == 1:
            data_df['HBV'][i] = 1
        else:
            data_df['HBV'][i] = 0
    if data_df['HBsAg'][i] == 1:
        data_df['HBV'][i] = 1
    elif data_df['HBsAg'][i] == 0:
        data_df['HBV'][i] = 0

In [30]:
# ****************************************
#只修正檢驗特徵，生理數值稍後處理
# ****************************************
# 抽血檢驗資料二次處理 - 轉成float type
# 處理非純數字的資料，把無法轉成數值的數據全改成9999
drop_cols = ['ID', '洗腎紀錄時間', '洗腎紀錄時間去時分', '急診門診住院','入院方式','開始血壓SBP', '開始血壓DBP', '開始脈搏', '體溫', '結束脈搏','Hollow fiber clot','處置其他結束']
for cols_2_num in data_df.columns.drop(drop_cols):
    data_df[cols_2_num] = pd.to_numeric(data_df[cols_2_num], errors='coerce')
    data_df[cols_2_num] = data_df[cols_2_num].replace(np.nan, -9999.0)
data_df.shape

(309556, 55)

# 生日、性別

In [31]:
data_df = data_df.merge(base_df, on=['ID'], how='left')
del(data_df['首次透析日期'])
data_df['算年齡'] = data_df['洗腎紀錄時間']
data_df['算年齡']= pd.to_datetime(data_df['算年齡'])
data_df['算年齡']= data_df['算年齡'].dt.date

In [32]:
print(len(set(data_df['ID'])))

1051


In [33]:
data_df['age'] = np.nan
for index in range(len(data_df)):
    data_df['age'][index] = math.floor((data_df['算年齡'][index] - data_df['生日'][index]).days/365.25)
# del(data_df['算年齡'])
# del(data_df['生日'])
data_df = data_df.replace('男', 1)
data_df = data_df.replace('女', 0)

# 體重調整

In [34]:
Peri_weight = pd.DataFrame()

Peri_weight["ID"], Peri_weight["洗腎紀錄時間去時分"] = peri_df["ID"], peri_df["洗腎紀錄時間"]
Peri_weight['體重1開始'], Peri_weight["體重2標準體重"],Peri_weight["體重結束"] = peri_df['體重1開始'], peri_df["體重2標準體重"], peri_df["體重結束"]
Peri_weight["體重實際脫水"], Peri_weight["體重機器顯示"], Peri_weight["體重3應脫水"] = peri_df["體重實際脫水"], peri_df["體重機器顯示"], peri_df["體重3應脫水"]

Peri_weight['洗腎紀錄時間去時分']= pd.to_datetime(Peri_weight['洗腎紀錄時間去時分'])
Peri_weight['洗腎紀錄時間去時分']= Peri_weight['洗腎紀錄時間去時分'].dt.date
print("============篩選前============")
print(Peri_weight.isnull().sum())
print(Peri_weight.shape)
Peri_weight = Peri_weight.drop_duplicates(keep='first', inplace=False) #假如有兩筆或數筆完整資料長一模一樣，則留第一筆資料(假定有資料重複紀錄)
# print(Peri_weight.shape)
Peri_weight = Peri_weight.drop_duplicates(subset=['ID','洗腎紀錄時間去時分'],keep='last', inplace=False) #假如有兩筆或數筆的ID和時間一樣，則保留最後一筆資料(假定第二次或最後一次才有正確記錄)
# print(Peri_weight.shape)
Peri_weight.index = range(len(Peri_weight))
print("============篩選後============")
print(Peri_weight.isnull().sum())
print(Peri_weight.shape)

============篩選前============
ID              0
洗腎紀錄時間去時分       0
體重1開始        1998
體重2標準體重       963
體重結束         3936
體重實際脫水       4059
體重機器顯示       3672
體重3應脫水       7451
dtype: int64
(309556, 8)
============篩選後============
ID              0
洗腎紀錄時間去時分       0
體重1開始        1307
體重2標準體重       790
體重結束         2448
體重實際脫水       2572
體重機器顯示       2203
體重3應脫水       6710
dtype: int64
(305543, 8)


In [35]:
print(len(set(data_df['ID'])))

1051


In [36]:
data_df['洗腎紀錄時間'] = pd.to_datetime(data_df['洗腎紀錄時間']).dt.date
data_df['洗腎紀錄時間去時分'] = pd.to_datetime(data_df['洗腎紀錄時間去時分']).dt.date
data_df['算年齡'] = pd.to_datetime(data_df['算年齡']).dt.date
Peri_weight[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']] = Peri_weight[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']].astype(str) #防止後續處理小數點時出現浮點數錯誤

In [37]:
data_df = pd.merge(data_df, Peri_weight, on=['ID','洗腎紀錄時間去時分'], how='left')
print(len(set(data_df['ID'])))

1051


In [38]:
if "第一階段處理結果-後半部檢驗資料[粗布處理體重]補值.csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[粗布處理體重]補值.csv")
else:
    # 修正字串轉數字步驟，原先做法容易讓脫水量的值出現異常
    # flist =['體重2標準體重','體重結束','體重實際脫水','體重機器顯示']
    flist =['體重1開始','體重2標準體重','體重結束']
    for f in flist:
        print(f)
        for i in tqdm(range(len(data_df))):
            if not pd.isnull(data_df[f][i]):
                data_df.loc[i,f] = ''.join([x for idx, x in enumerate(data_df[f][i]) if x.isdigit() or x=='.' or (idx==0 and x=='-')]) 
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[粗布處理體重]補值.csv", index=False, encoding='utf-8-sig')
data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']] = data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']].astype(str) #防止後續處理小數點時出現浮點數錯誤


體重1開始


  0%|          | 0/309556 [00:00<?, ?it/s]

體重2標準體重


  0%|          | 0/309556 [00:00<?, ?it/s]

體重結束


  0%|          | 0/309556 [00:00<?, ?it/s]

In [39]:
data_df['體重1開始'] = pd.to_numeric(data_df['體重1開始'], errors='coerce')
data_df['體重1開始'] = data_df['體重1開始'].fillna(-9999)
data_df['體重1開始'] = data_df['體重1開始'].apply(lambda x: 19999 if x==0 else x)

In [40]:
if "第一階段處理結果-後半部檢驗資料[1-開始體重]補值.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[1-開始體重]補值.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[1-開始體重]補值.csv", index=False, encoding='utf-8-sig')
data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']] = data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']].astype(str) #防止後續處理小數點時出現浮點數錯誤

In [41]:
# 把無法轉換成數字的資料全改成9999
# 把體重等於0的數據改成9999
data_df['體重2標準體重'] = pd.to_numeric(data_df['體重2標準體重'], errors='coerce')
data_df['體重2標準體重'] = data_df['體重2標準體重'].fillna(-9999)
data_df['體重2標準體重'] = data_df['體重2標準體重'].apply(lambda x: 19999 if x==0 else x)

In [42]:
if "第一階段處理結果-後半部檢驗資料[2-乾體重]補值.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[2-乾體重]補值.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[2-乾體重]補值.csv", index=False, encoding='utf-8-sig')
data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']] = data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','體重3應脫水']].astype(str) #防止後續處理小數點時出現浮點數錯誤

In [43]:
# 把無法轉換成數字的資料全改成9999
# 把體重等於0的數據改成9999
data_df['體重結束'] = pd.to_numeric(data_df['體重結束'], errors='coerce')
data_df['體重結束'] = data_df['體重結束'].fillna(-9999)
data_df['體重結束'] = data_df['體重結束'].apply(lambda x: 19999 if x==0 else x)

In [44]:
if "第一階段處理結果-後半部檢驗資料[3-體重結束]補值.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[3-體重結束]補值.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[3-體重結束]補值.csv", index=False, encoding='utf-8-sig')
data_df[['體重1開始','體重2標準體重','體重結束']] = data_df[['體重1開始','體重2標準體重','體重結束']].astype(float) #防止後續處理小數點時出現浮點數錯誤

### 粗步修正實際脫水量

In [45]:
# 把無法轉換成數字的資料全改成9999
data_df['體重實際脫水'] = pd.to_numeric(data_df['體重實際脫水'], errors='coerce')
data_df['體重實際脫水'] = data_df['體重實際脫水'].fillna(-9999)
data_df['體重實際脫水'] = data_df['體重實際脫水'].apply(lambda x: 0 if x<0 else x)

In [46]:
if "第一階段處理結果-後半部檢驗資料[4-體重實際脫水]補值.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[4-體重實際脫水]補值.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[4-體重實際脫水]補值.csv", index=False, encoding='utf-8-sig')
data_df[['體重1開始','體重2標準體重','體重結束']] = data_df[['體重1開始','體重2標準體重','體重結束']].astype(str) #防止後續處理小數點時出現浮點數錯誤

### 粗步修正機器顯示脫水量

In [47]:
# 把無法轉換成數字的資料全改成9999
data_df['體重機器顯示'] = pd.to_numeric(data_df['體重機器顯示'], errors='coerce')
data_df['體重機器顯示'] = data_df['體重機器顯示'].fillna(-9999)
data_df['體重機器顯示'] = data_df['體重機器顯示'].apply(lambda x: 0 if x<0 else x)

In [48]:
if "第一階段處理結果-後半部檢驗資料[5-體重機器顯示]補值.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[5-體重機器顯示]補值.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[5-體重機器顯示]補值.csv", index=False, encoding='utf-8-sig')

In [49]:
data_df[['體重2標準體重', '體重1開始','體重結束','體重機器顯示','體重實際脫水']].astype(float).describe()

,體重2標準體重,體重1開始,體重結束,體重機器顯示,體重實際脫水
count,309556.000000,309556.000000,309556.000000,309556.000000,309556.000000
mean,-57.313208,44.948736,-70.047908,2.337509,2.302423
std,1127.371689,1244.151973,1377.752947,1.423729,2.694028
min,-9999.000000,-9999.000000,-9999.000000,0.000000,0.000000
25%,52.100000,54.600000,52.100000,1.600000,1.600000
50%,60.700000,63.300000,60.700000,2.200000,2.200000
75%,68.500000,71.500000,68.500000,3.000000,2.900000
max,19999.000000,19999.000000,420001.000000,188.000000,1007.000000


### 粗步計算每公斤脫水量

In [50]:

# 粗布推估脫水量一率以「體重實際脫水」為主
# 有負數則改成9999
weight_water_list = data_df[['體重1開始', '體重結束', '體重機器顯示', '體重實際脫水']].astype(float)
# weight_water_list['8%檢測值'] = (weight_water_list['體重1開始']*0.08)
# weight_water_list['[開始體重-結束體重]'] = weight_water_list['體重1開始'] - weight_water_list['體重結束']
# print(weight_water_list.eq(9999.0).sum())
# new_real_water_df = []
# for idx, value in tqdm(weight_water_list.iterrows()):
#     se = value['[開始體重-結束體重]']
#     eight = value['8%檢測值']
#     if value['體重實際脫水']<=eight and value['體重實際脫水']<=10:
#         new_real_water_df.append(value['體重實際脫水'])
#     elif value['體重機器顯示']<=eight and value['體重機器顯示']<=10:
#         new_real_water_df.append(value['體重機器顯示'])
#     elif se<=eight and se<=10 and (value['體重1開始']!=9999 and value['體重結束']!=9999.0):
#         # print(value['體重1開始'], value['體重結束'])
#         new_real_water_df.append(se)
#     else:
#         new_real_water_df.append(9999.0)
#     if new_real_water_df[-1]<0:
#         # print(value['體重1開始'], value['體重結束'],new_real_water_df[-1])
#         new_real_water_df[-1] = 9999.0
# data_df['體重實際脫水'] = new_real_water_df
# print(weight_water_list.eq(9999.0).sum())

In [51]:
if "第一階段處理結果-後半部檢驗資料[6-體重實際脫水]補值.csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[6-體重實際脫水]補值.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[6-體重實際脫水]補值.csv", index=False, encoding='utf-8-sig')

# 各項體重修正 - 開始體重、乾體重、結束體重、實際脫水量

In [52]:
data_df[['體重1開始', '體重2標準體重','體重結束', '體重實際脫水', '體重機器顯示']] = data_df[['體重1開始', '體重2標準體重','體重結束', '體重實際脫水', '體重機器顯示']].astype(float)

In [53]:
weight_check = data_df[['ID', '洗腎紀錄時間','體重1開始', '體重2標準體重','體重結束', '體重實際脫水', '體重機器顯示']]
weight_check['開始-結束體重'] = [19999.0 if (cols['體重1開始'] in [-9999, 19999]) or (cols['體重結束'] in [-9999, 19999]) else cols['體重1開始'] - cols['體重結束'] for idx, cols in weight_check.iterrows()]
# 確認分母體重還有多少筆9999.0
print(len(set(data_df['ID'])))

1051


In [54]:
weight_check.describe()

,體重1開始,體重2標準體重,體重結束,體重實際脫水,體重機器顯示,開始-結束體重
count,309556.000000,309556.000000,309556.000000,309556.000000,309556.000000,309556.000000
mean,44.948736,-57.313208,-70.047908,2.302423,2.337509,317.602700
std,1244.151973,1127.371689,1377.752947,2.694028,1.423729,2609.270817
min,-9999.000000,-9999.000000,-9999.000000,0.000000,0.000000,-419956.400000
25%,54.600000,52.100000,52.100000,1.600000,1.600000,1.700000
50%,63.300000,60.700000,60.700000,2.200000,2.200000,2.300000
75%,71.500000,68.500000,68.500000,2.900000,3.000000,3.100000
max,19999.000000,19999.000000,420001.000000,1007.000000,188.000000,19999.000000


### 創建分母體重

In [55]:
# 數據B、C移除分母體重
# # 分母體重 處理成透析成體重
# # 分母體重篩選流程：開始體重->結束體重->乾體重->9999
# # 以體重差做評估: 處理合理範圍但實際邏輯不合理的數值，洗前(體重1開始)-洗後的差區間不符合-5~10kg
# # 在任一體重或脫水量非9999情況之下評估abs(開始體重-乾體重)、abs(結束體重-乾體重)哪一個差最小
# # 如(結束體重-乾體重)差最小，則以(結束體重+實際脫水)視為新的分母體重
# weight_check['分母體重'] = np.nan
# weight_select_df = []
# for index, weight_rows in tqdm(weight_check.iterrows()):
#     if weight_rows['體重1開始']!=9999:
#         weight_check.loc[index, '分母體重'] = weight_rows['體重1開始']
#         weight_select_df.append(0)
#     elif weight_rows['體重結束']!=9999:
#         weight_check.loc[index, '分母體重'] = weight_rows['體重結束']
#         weight_select_df.append(1)
#     elif weight_rows['體重2標準體重']!=9999:
#         weight_check.loc[index, '分母體重'] = weight_rows['體重2標準體重']
#         weight_select_df.append(2)
#     else:
#         weight_check.loc[index, '分母體重'] = 9999.0
#         weight_select_df.append(3)

In [56]:
# # 體重修正統計分佈圖：評估分母體重的來源體重比例和分佈
# temp_null = np.unique(np.array(weight_select_df), return_counts = True)
# print(temp_null)
# x = ['Start Weight','End Weight','Dry Weight','Null']
# h = temp_null[1]
# sns.barplot(x=x, y=h)
# for i in range(len(x)):
#     plt.text(x=i, y=h[i]+1, s=str(h[i]), ha='center', va='bottom')
# plt.xlabel('weight select list')
# plt.ylabel('total count')
# plt.title("weight adjust distributed")
# plt.show()

### 新增分母體重修正實際脫水量

In [57]:
# # 分母體重 處理成透析成體重
# # 分母體重篩選流程：開始體重->結束體重->乾體重->9999
# # 以體重差做評估: 處理合理範圍但實際邏輯不合理的數值，洗前(體重1開始)-洗後的差區間不符合-5~10kg
# # 在任一體重或脫水量非9999情況之下評估abs(開始體重-乾體重)、abs(結束體重-乾體重)哪一個差最小
# # 如(結束體重-乾體重)差最小，則以(結束體重+實際脫水)視為新的分母體重
# # 根據體重差二次評估分母體重的可靠性
# check_num = 0
# for index, weight_row in tqdm(weight_check.iterrows()):
#     if (-5<=weight_row['開始-結束體重']<=10) ==False and (weight_row['體重1開始']!=9999.0 and weight_row['體重結束']!=9999.0 and weight_row['體重實際脫水']!=9999.0):
#         sd = abs(weight_row['體重1開始']-weight_row['體重2標準體重'])
#         ed = abs(weight_row['體重結束']-weight_row['體重2標準體重'])
#         if sd>ed:
#             weight_check.loc[index,'分母體重'] = weight_row['體重實際脫水'] + weight_row['體重結束']
#             check_num+=1
# print(f"共修正{check_num}筆分母體重")
# # 共修正2700筆分母體重

In [58]:
# data_df[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','分母體重']] = weight_check[['體重1開始','體重2標準體重','體重結束','體重實際脫水','體重機器顯示','分母體重']]
# print(f"分母體重共{len(data_df[data_df['分母體重']==9999.0])}筆異常資料")
# data_df.describe()

In [59]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
index,309556.0,179308.629007,103597.829404,4.0,90895.750,180416.500,268817.25000,358822.00
入院方式,309556.0,-75.957387,870.305905,-9999.0,0.000,0.000,0.00000,3.00
FISTULA,309556.0,0.840026,0.366583,0.0,1.000,1.000,1.00000,1.00
GRAFT,309556.0,0.130632,0.336998,0.0,0.000,0.000,0.00000,1.00
DLC,309556.0,0.000853,0.029191,0.0,0.000,0.000,0.00000,1.00
PERMCATH,309556.0,0.066660,0.249433,0.0,0.000,0.000,0.00000,1.00
Catheter,309556.0,0.066812,0.249696,0.0,0.000,0.000,0.00000,1.00
K,309556.0,2.171039,149.334299,-9999.0,3.900,4.400,4.80000,21.30
HGB,309556.0,10.407990,84.396585,-9999.0,10.300,11.200,12.00000,17.40
HCT,309556.0,32.580288,119.683641,-9999.0,31.500,34.100,36.70000,56.30


In [60]:
#體重修正
weight_check = lambda x: 20<x<150

for weight  in ['體重1開始', '體重2標準體重', '體重結束']:
    data_df[weight] = data_df[weight].apply(lambda x: x if (weight_check(x)==True) or (x==-9999) else 19999)

In [61]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
index,309556.0,179308.629007,103597.829404,4.0,90895.750,180416.500,268817.25000,358822.00
入院方式,309556.0,-75.957387,870.305905,-9999.0,0.000,0.000,0.00000,3.00
FISTULA,309556.0,0.840026,0.366583,0.0,1.000,1.000,1.00000,1.00
GRAFT,309556.0,0.130632,0.336998,0.0,0.000,0.000,0.00000,1.00
DLC,309556.0,0.000853,0.029191,0.0,0.000,0.000,0.00000,1.00
PERMCATH,309556.0,0.066660,0.249433,0.0,0.000,0.000,0.00000,1.00
Catheter,309556.0,0.066812,0.249696,0.0,0.000,0.000,0.00000,1.00
K,309556.0,2.171039,149.334299,-9999.0,3.900,4.400,4.80000,21.30
HGB,309556.0,10.407990,84.396585,-9999.0,10.300,11.200,12.00000,17.40
HCT,309556.0,32.580288,119.683641,-9999.0,31.500,34.100,36.70000,56.30


In [62]:
if "第一階段處理結果-後半部檢驗資料[7-檢驗值確認結束]計算.csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[7-檢驗值確認結束]計算.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[7-檢驗值確認結束]計算.csv", index=False, encoding='utf-8-sig')

### 二次修正每公斤脫水量

In [63]:
# 每公斤脫水量計算
weight_water = data_df[['體重1開始', '體重實際脫水']]
kg_water = []
for idx, value in tqdm(weight_water.iterrows()):
    # if (value['分母體重']!=9999 and value['體重實際脫水']!=9999):
        kg_water.append((value['體重實際脫水']*1000)/value['體重1開始'])
    # else:
    #     kg_water.append(9999.0)
weight_water['每公斤脫水量(ml/kg)'] = kg_water
# 如出現大於9999的脫水量，則改為9999
weight_water['每公斤脫水量(ml/kg)'] = weight_water['每公斤脫水量(ml/kg)'].apply(lambda x: 19999 if x>100 else x)
data_df['每公斤脫水量(ml/kg)'] = weight_water['每公斤脫水量(ml/kg)']
data_df['每公斤脫水量(ml/kg)'] = data_df['每公斤脫水量(ml/kg)'].apply(lambda x: 19999 if x<=0 else x)
weight_water.describe()

0it [00:00, ?it/s]

,體重1開始,體重實際脫水,每公斤脫水量(ml/kg)
count,309556.000000,309556.000000,309556.000000
mean,52.739320,2.302423,54.682022
std,1307.222473,2.694028,623.419654
min,-9999.000000,0.000000,-0.500050
25%,54.700000,1.600000,26.052104
50%,63.300000,2.200000,35.087719
75%,71.600000,2.900000,44.515103
max,19999.000000,1007.000000,19999.000000


In [64]:
if "第一階段處理結果-後半部檢驗資料[8-每公斤脫水量]計算.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[8-每公斤脫水量]計算.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[8-每公斤脫水量]計算.csv", index=False, encoding='utf-8-sig')

In [65]:
# weight_list = data_df[['體重1開始', '體重2標準體重', '體重結束', '分母體重', '體重機器顯示', '體重實際脫水']]
# # 體重評估
# for cols in weight_list.columns:
#     print(f"[[{cols }]]")
#     weight__ = weight_list[cols].values
#     weight__2 = weight__[weight__!=9999]
#     if cols in ['體重1開始', '體重2標準體重', '體重結束', '分母體重']:
#         weight__3 = weight__[(weight__<20)|(weight__>150)]
#         print(f"異常筆數: {len(weight__3)}")
#     else:
#         weight__3 = weight__[(weight__<0.0)|(weight__>10)]
#         print(f"異常筆數: {len(weight__3)}")
#     # print(f"Null筆數: {len(weight__)-len(weight__2)}")
#     # print(f"{cols } Min: {np.min(weight__2):.2f} Max: {np.max(weight__2):.2f} Mean: {np.mean(weight__2):.2f}")
#     print(f"Min: {np.min(weight__2):.2f} Max: {np.max(weight__2):.2f}\n")
#     # break
# weight_list['驗證：開始-結束'] = weight_list['體重1開始']-weight_list['體重結束']
# weight_list['驗證：開始8%'] = weight_list['體重1開始']*0.08
# weight_list['驗證：開始-結束'] = weight_list['驗證：開始-結束'].apply(lambda x: 0.0 if x<0 else x)
# print("共{}筆資料大於10公斤脫水量".format(weight_list['驗證：開始-結束'].apply(lambda x: 1 if x>10 else 0).sum()))
# print("共{}筆資料大於8%基底體重脫水量".format(np.sum([weight_list['驗證：開始8%']<weight_list['驗證：開始-結束']])))

# 增加糖尿病

In [66]:
#判斷有無糖尿病
OPD = pd.read_csv("./dataset/drug_lab_base_order/OPD_en_utf8.txt")
lab_df_Anti = lab_df_Anti[lab_df_Anti['檢驗名稱'] == 'HbA1c']
lab_df_Anti = lab_df_Anti[lab_df_Anti['DATANUM'] > '6.5']

In [67]:
# 新增一個表格df 判斷這個ID是不是有糖尿病
df =  pd.DataFrame({'ID':data_df['ID']})
df = df.drop_duplicates(keep='first', inplace=False)
df.index = range(len(df))

In [68]:
#初步用HbA1c是否大於6.5的資料判斷有沒有糖尿病
df['有無糖尿病值'] = np.nan
for i in range(len(df)): 
    name = df['ID'][i]
    feature_df = lab_df_Anti [lab_df_Anti ['ID'] == name]
    if len(feature_df) > 0:
        df['有無糖尿病值'][i] = 1
    else:
        df['有無糖尿病值'][i] = 0

In [69]:
OPD =  pd.DataFrame({'ID':OPD['ID'], 'CHOP1ICD1':OPD['CHOP1ICD1'], 'CHOP1ICD2':OPD['CHOP1ICD2'], 'CHOP1ICD3':OPD['CHOP1ICD3'], 'CHOP1ICD4':OPD['CHOP1ICD4'], 'CHOP1ICD5':OPD['CHOP1ICD5'], 'CHOP1ICD6':OPD['CHOP1ICD6']})
OPD = OPD.fillna('0')

In [70]:
#用診斷碼的資料判斷有沒有糖尿病
OPD['有無糖尿病診斷碼'] = np.nan
for i in range(len(OPD)):
    if ('250.' in OPD['CHOP1ICD1'][i]) | ('250.' in OPD['CHOP1ICD2'][i]) | ('250.' in OPD['CHOP1ICD3'][i])\
    | ('250.' in OPD['CHOP1ICD4'][i]) | ('250.' in OPD['CHOP1ICD5'][i]) | ('250.' in OPD['CHOP1ICD6'][i]):
        OPD['有無糖尿病診斷碼'][i] = 1
        
    elif ('357.2' in OPD['CHOP1ICD1'][i]) | ('357.2' in OPD['CHOP1ICD2'][i]) | ('357.2' in OPD['CHOP1ICD3'][i])\
    | ('357.2' in OPD['CHOP1ICD4'][i]) | ('357.2' in OPD['CHOP1ICD5'][i]) | ('357.2' in OPD['CHOP1ICD6'][i]):
        OPD['有無糖尿病診斷碼'][i] = 1
        
    elif ('326.0' in OPD['CHOP1ICD1'][i]) | ('326.0' in OPD['CHOP1ICD2'][i]) | ('326.0' in OPD['CHOP1ICD3'][i])\
    | ('326.0' in OPD['CHOP1ICD4'][i]) | ('326.0' in OPD['CHOP1ICD5'][i]) | ('326.0' in OPD['CHOP1ICD6'][i]):
        OPD['有無糖尿病診斷碼'][i] = 1
        
    elif ('362.07' in OPD['CHOP1ICD1'][i]) | ('362.07' in OPD['CHOP1ICD2'][i]) | ('362.07' in OPD['CHOP1ICD3'][i])\
    | ('362.07' in OPD['CHOP1ICD4'][i]) | ('362.07' in OPD['CHOP1ICD5'][i]) | ('362.07' in OPD['CHOP1ICD6'][i]):
        OPD['有無糖尿病診斷碼'][i] = 1
        
    else:
        OPD['有無糖尿病診斷碼'][i] = 0

In [71]:
OPD = pd.DataFrame({'ID':OPD['ID'], '有無糖尿病診斷碼':OPD['有無糖尿病診斷碼']})
OPD = OPD.drop_duplicates(keep='first', inplace=False)
OPD.index = range(len(OPD))
OPD_1 = pd.DataFrame(columns = ['ID', '有無糖尿病診斷碼'])

In [72]:
for i in set(OPD['ID']):
    O = OPD[OPD['ID'] == i]
    if len(O) == 1:
        new_list = [(i, O['有無糖尿病診斷碼'].values)]
    else:
        O1 = O[O['有無糖尿病診斷碼'] == 1]
        new_list = [(i, O1['有無糖尿病診斷碼'].values)]
    dfNew=pd.DataFrame(new_list, columns = ['ID' , '有無糖尿病診斷碼'])
    # OPD_1=OPD_1.append(dfNew,ignore_index=True)
    OPD_1 = pd.concat((OPD_1, dfNew))

In [73]:
OPD_1['有無糖尿病診斷碼0'] = OPD_1['有無糖尿病診斷碼'].apply(lambda x: ', '.join([str(i) for i in x]))
del(OPD_1['有無糖尿病診斷碼'])
OPD_1 = OPD_1.drop_duplicates(keep='first', inplace=False)
OPD_1.index = range(len(OPD_1))
df = pd.merge(df, OPD_1 , on=['ID'], how='left')

In [74]:
df['有無糖尿病'] = np.nan
for i in range(len(df)):
    if (df['有無糖尿病值'][i] == 1) | (df['有無糖尿病診斷碼0'][i] == '1.0'):
        df['有無糖尿病'][i] = 1
        
    else:
        df['有無糖尿病'][i] = 0
del(df['有無糖尿病診斷碼0'])
del(df['有無糖尿病值'])
# 將判斷好的ＩＤ是否有糖尿病按照ＩＤmerge到df裏
data_df = pd.merge(data_df, df, on=['ID'], how='left')

In [75]:
if "第一階段處理結果-後半部檢驗資料[9-糖尿病]計算.csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第一階段處理結果-後半部檢驗資料[9-糖尿病]計算.csv")
else:
    data_df.to_csv("./dataset/第一階段處理結果-後半部檢驗資料[9-糖尿病]計算.csv", index=False, encoding='utf-8-sig')

In [76]:
print(len(set(data_df['ID'])))

1051


# 第二階段處理
## 處理新的特徵、血壓、體溫、脈搏

In [77]:
# data_df['開始脈搏'].unique()

In [78]:
flist =['開始血壓SBP', '開始血壓DBP', '開始脈搏','體溫', '結束脈搏']
if "第二階段處理結果-生理數據[1-血壓脈搏粗布處裡]修正.csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第二階段處理結果-生理數據[1-血壓脈搏粗布處裡]修正.csv")
else:
    # 修正字串轉數字步驟
    flist =['開始血壓SBP', '開始血壓DBP', '開始脈搏','體溫', '結束脈搏']
    data_df[flist] = data_df[flist].astype(str) #防止後續處理小數點時出現浮點數錯誤
    for f in flist:
        print(f)
        for i in tqdm(range(len(data_df))):
            if not pd.isnull(data_df[f][i]):
                data_df.loc[i,f] = ''.join([x for idx, x in enumerate(data_df[f][i]) if x.isdigit() or x=='.' or x=='-' or x=='/']) 
    data_df.to_csv("./dataset/第二階段處理結果-生理數據[1-血壓脈搏粗布處裡]修正.csv", index=False, encoding='utf-8-sig')

開始血壓SBP


  0%|          | 0/309556 [00:00<?, ?it/s]

開始血壓DBP


  0%|          | 0/309556 [00:00<?, ?it/s]

開始脈搏


  0%|          | 0/309556 [00:00<?, ?it/s]

體溫


  0%|          | 0/309556 [00:00<?, ?it/s]

結束脈搏


  0%|          | 0/309556 [00:00<?, ?it/s]

In [79]:
# 30 <= Pulse <= 150 # 40 <= SBP <= 240 # 30 <= DBP <= 140
# *****2024.04.26修正：合理SBP範圍更新為60-240*****

In [80]:
from utils.value_check_miss import *

In [81]:
# 初步字串轉數值
# 起初SBP血壓如果過低會多測量幾次
data_df[flist] = data_df[flist].astype(str) #防止後續處理小數點時出現浮點數錯誤
data_df['開始血壓SBP'] = data_df['開始血壓SBP'].apply(lambda x: x.replace("..",".") if ".." in x else x)
data_df['開始血壓SBP'].replace(['-','nan','Nan','',"--",],"-9999.0", inplace=True)
# 將有範圍且符合血壓範圍的資料做平均補值, e.g: 83-106 => 94
data_df['開始血壓SBP'] = data_df['開始血壓SBP'].apply(lambda x: pulse_process(x, type_='sbp'))
for index, bp in (data_df[['ID','開始血壓SBP']].iterrows()):
    if str(bp['開始血壓SBP']).count('-') ==1:
        bp_split = str(bp['開始血壓SBP']).split('-')
        if bp_split[0]!='' and bp_split[-1]!='':
            # if (40 <= int(bp_split[0]) <= 240) and (40 <= int(bp_split[-1]) <= 240):
            if (60 <= int(bp_split[0]) <= 240) and (60 <= int(bp_split[-1]) <= 240):
                print(bp_split)
                data_df.loc[index,'開始血壓SBP'] = (int(bp_split[0])+int(bp_split[-1]))//2
            
data_df['開始血壓SBP'] = pd.to_numeric(data_df['開始血壓SBP'])
# data_df['開始血壓SBP'].replace(9999, 19999, inplace=True)

In [82]:
data_df['開始血壓DBP'] = data_df['開始血壓DBP'].apply(lambda x: x.replace("..",".") if ".." in x else x)
data_df['開始血壓DBP'].replace(['--','-','nan','Nan',''],"-9999", inplace=True)
data_df['開始血壓DBP'] = data_df['開始血壓DBP'].apply(lambda x: pulse_process(x, type_='dbp'))
# 將有範圍且符合血壓範圍的資料做平均補值, e.g: 83-106 => 94
for index, bp in (data_df[['ID','開始血壓DBP']].iterrows()):
    if str(bp['開始血壓DBP']).count('-') >0:
        bp_split = str(bp['開始血壓DBP']).split('-')
        if bp_split[0]!='' and bp_split[-1]!='':
            if (30 <= int(bp_split[0]) <= 140) and (30 <= int(bp_split[-1]) <= 140):
                print(bp_split)
                data_df.loc[index,'開始血壓DBP'] = (int(bp_split[0])+int(bp_split[-1]))//2

data_df['開始血壓DBP'] = pd.to_numeric(data_df['開始血壓DBP'])
# data_df['開始血壓DBP'].replace(9999, 19999, inplace=True)

In [83]:
data_df['開始脈搏'] = pd.to_numeric(data_df['開始脈搏'], errors='coerce')
data_df['開始脈搏'] = data_df['開始脈搏'].fillna(-9999)
data_df['開始脈搏'] = data_df['開始脈搏'].apply(lambda x: 19999 if x==0 else x)

In [84]:
data_df['結束脈搏'] = pd.to_numeric(data_df['結束脈搏'], errors='coerce')
data_df['結束脈搏'] = data_df['結束脈搏'].fillna(-9999)
data_df['結束脈搏'] = data_df['結束脈搏'].apply(lambda x: 19999 if x==0 else x)
data_df['結束脈搏'] = pd.to_numeric(data_df['結束脈搏'])

In [85]:
data_df['體溫'] = pd.to_numeric(data_df['體溫'], errors='coerce')
data_df['體溫'] = data_df['體溫'].fillna(-9999)
data_df['體溫'] = data_df['體溫'].apply(lambda x: 19999 if x==0 else x)
data_df['體溫'] = pd.to_numeric(data_df['體溫'])

In [86]:
if "第二階段處理結果-生理數據[2-修正處理].csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第二階段處理結果-生理數據[2-修正處理].csv")
else:
    data_df.to_csv("./dataset/第二階段處理結果-生理數據[2-修正處理].csv", index=False, encoding='utf-8-sig')

In [87]:
#添加原始index，用於追蹤後續資料用
data_df.insert(0, 'Raw Index', list(data_df.index)) 

# 處理IntraHD資料 - 修正原始血壓異常值

In [88]:
# loading intraHD 
# 處理intraHD血壓
# intra_df = pd.read_csv("./dataset/drug_lab_base_order/intraHD2016_20_en.csv") old
intra_df_1 = pd.read_csv('./dataset/drug_lab_base_order/intraHD2016_18_utf8.txt')
intra_df_2 = pd.read_csv('./dataset/drug_lab_base_order/intraHD2019_20_utf8.txt')
intra_df = pd.concat([intra_df_1, intra_df_2])
del intra_df_1, intra_df_2
intra_df['洗腎紀錄時間']= pd.to_datetime(intra_df['洗腎紀錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間']
intra_df['中記錄時間']= pd.to_datetime(intra_df['中記錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間去時分'].dt.date

In [89]:
intra_df['血壓'] = intra_df['血壓'].apply(lambda x: x[1::].replace('.','/') if x[0]=='/' and '.' in x else x) # e.g: /144.62
intra_df['血壓'] = intra_df['血壓'].apply(lambda x: x[1::].replace('/.','/').replace('./','/') if '/.' in x or './' in x else x) # e.g: 144./62、144/.62
intra_df['血壓'] = intra_df['血壓'].apply(lambda x: x.replace('.','') if '.' in x else x) # e.g 14.3/84、.109/59、123/63.
#當前資料狀況基本上血壓都會有/作為區分sbp/dbp，以下為異常資料清單
# 'May-81', 'Dec-54', 'Jun-97', 'May-95', '14      ', '10月26日', 'Jun-96', 'Jun-93', 'May-94', 'Mar-72', 'May-78', 'Apr-72', 'Apr-69', 'Apr-83', 'May-99', 'Sep-99', '9月20日', '2月19日', 'May-95', 'Apr-93', 'May-90', 'May-84', 'Mar-91', 'Jun-97', '7月17日', 'May-86', 'Nov-95', 'Jun-96', '9月13日', 'May-95', 'Jul-89', 'May-88', 'May-96', 'May-86', 'Mar-68', 'Apr-96', '5月16日', '1.7', 'Jun-95', '11月3日', '11月6日', 'Nov-69', 'May-95', 'Apr-58', '-1.771428571', '-1.771428571', 'Jun-92', '10月10日', '10月10日', 'Apr-89', 'Apr-97', 'Mar-96', '1月1日', '1月1日', '1月1日', '-2', '1月1日', 'May-90', '1.638554217', 'Jun-97', '1月1日', '1月1日', '1月1日', '1月1日', '1月1日', '1月1日', 'Nov-49', '7月3日', 'Oct-86', 'Feb-73', '12月29日', 'Mar-75', '1月2日', '1月1日', '1月1日', '11月11日', '2月2日'
intra_df['血壓'] = intra_df['血壓'].apply(lambda x: re.sub(r'[^0-9\/]', '', x))
intra_df['血壓'] = intra_df['血壓'].apply(lambda x: '-9999/-9999' if '/' not in x else x)
intra_df['血壓'] = intra_df['血壓'].apply(lambda x: '-9999/-9999' if x=='/' else x)

In [90]:
sbp_list, dbp_list = [], []
for i in intra_df['血壓']:
    bp = i.split('/')
    if len(bp)==2: #有些紀錄有三筆數字。 e.g: 144/44/65
        sbp_list.append(bp[0]); dbp_list.append(bp[1])
    else:
        sbp_list.append(19999); dbp_list.append(19999)
intra_df['sbp'],intra_df['dbp'] = sbp_list, dbp_list
intra_df['sbp'],intra_df['dbp'] = pd.to_numeric(intra_df['sbp']), pd.to_numeric(intra_df['dbp'])
intra_df.to_csv('./dataset/drug_lab_base_order/intraHD2016_20_fix_2024-09-25.csv', encoding='utf-8-sig', index=False)

### 粗布處理血壓異常範圍

In [91]:
# loading intraHD 
intra_df = pd.read_csv("./dataset/drug_lab_base_order/intraHD2016_20_fix_2024-04-29.csv")
intra_df['洗腎紀錄時間']= pd.to_datetime(intra_df['洗腎紀錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間']
intra_df['中記錄時間']= pd.to_datetime(intra_df['中記錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間去時分'].dt.date
intra_df[intra_df['急診門診住院']=='2.門診  ']
data_df['洗腎紀錄時間去時分']= pd.to_datetime(data_df['洗腎紀錄時間去時分']).dt.date

In [92]:
# 修正outlier 開始血壓SBP、DBP
# 40 <= SBP <= 240 # 30 <= DBP <= 140
# *****2024.04.26修正：合理SBP範圍更新為60-240*****
# data_df['開始血壓SBP'] = data_df['開始血壓SBP'].apply(lambda x: 9999.0 if x>240 or x <40 else x)
data_df['開始血壓SBP'] = data_df['開始血壓SBP'].apply(lambda x: 19999.0 if (x>240 or x <60) and x!=-9999 else x)
data_df['開始血壓DBP'] = data_df['開始血壓DBP'].apply(lambda x: 19999.0 if (x>140 or x <30) and x!=-9999 else x)
for cols in ['開始血壓DBP','開始血壓SBP']:
    temp_ = data_df[(data_df[cols]!=-9999)&(data_df[cols]!=19999)][cols]
    print(f"Max:{float(min(temp_))} \t Min:{float(max(temp_))}\t- {cols}")

# error_startSBP = data_df[((data_df['開始血壓SBP']>240)|(data_df['開始血壓SBP']<40))][['ID','洗腎紀錄時間','洗腎紀錄時間去時分','開始血壓SBP']]
error_startSBP = data_df[((data_df['開始血壓SBP']>240)|(data_df['開始血壓SBP']<60))][['ID','洗腎紀錄時間','洗腎紀錄時間去時分','開始血壓SBP']]
print(f"共計{error_startSBP.shape[0]}筆錯誤SBP")

error_startDBP = data_df[((data_df['開始血壓DBP']>140)|(data_df['開始血壓DBP']<30))][['ID','洗腎紀錄時間','洗腎紀錄時間去時分','開始血壓DBP']]
print(f"共計{error_startDBP.shape[0]}筆錯誤DBP")

for index, error_row in tqdm(error_startSBP.iterrows()):
    match_intra_df = intra_df[(intra_df['ID']==error_row['ID'])&(intra_df['洗腎紀錄時間去時分']==error_row['洗腎紀錄時間去時分'])]
    match_intra_df = match_intra_df.sort_values('中記錄時間')
    if len(match_intra_df)!=0:
        intra_first_bp = match_intra_df.loc[match_intra_df.index[0]]['sbp']
        # if 40 <= intra_first_bp <= 240:
        if 60 <= intra_first_bp <= 240:
            error_startSBP.loc[index,'開始血壓SBP'] = intra_first_bp
        else:
            error_startSBP.loc[index,'開始血壓SBP'] = 19999.0
    else:
        error_startSBP.loc[index,'開始血壓SBP'] = -9999.0
data_df.update(error_startSBP)
print(f"19999 共計{data_df[data_df['開始血壓SBP']==19999].shape[0]}筆錯誤SBP")
print(f"-9999 共計{data_df[data_df['開始血壓SBP']==-9999].shape[0]}筆錯誤SBP")

for index, error_row in tqdm(error_startDBP.iterrows()):
    match_intra_df = intra_df[(intra_df['ID']==error_row['ID'])&(intra_df['洗腎紀錄時間去時分']==error_row['洗腎紀錄時間去時分'])]
    match_intra_df = match_intra_df.sort_values('中記錄時間')
    if len(match_intra_df)!=0:
        intra_first_bp = match_intra_df.loc[match_intra_df.index[0]]['dbp']
        if 30 <= intra_first_bp <= 140:
            error_startDBP.loc[index,'開始血壓DBP'] = intra_first_bp
        else:
            error_startDBP.loc[index,'開始血壓DBP'] = 19999.0
    else:
        error_startDBP.loc[index,'開始血壓DBP'] = -9999.0
data_df.update(error_startDBP)
print(f"19999 共計{data_df[data_df['開始血壓DBP']==19999].shape[0]}筆錯誤DBP")
print(f"-9999 共計{data_df[data_df['開始血壓DBP']==-9999].shape[0]}筆錯誤DBP")

Max:30.0 	 Min:140.0	- 開始血壓DBP
Max:60.0 	 Min:240.0	- 開始血壓SBP
共計3035筆錯誤SBP
共計2855筆錯誤DBP


0it [00:00, ?it/s]

19999 共計266筆錯誤SBP
-9999 共計545筆錯誤SBP


0it [00:00, ?it/s]

19999 共計213筆錯誤DBP
-9999 共計534筆錯誤DBP


In [93]:
if "第二階段處理結果-生理數據[3-血壓補值處裡].csv" in os.listdir("./dataset"):
    data_df = pd.read_csv("./dataset/第二階段處理結果-生理數據[3-血壓補值處裡].csv")
else:
    data_df.to_csv("./dataset/第二階段處理結果-生理數據[3-血壓補值處裡].csv", index=False, encoding='utf-8-sig')

### 粗布處理脈搏異常範圍

In [94]:
# loading intraHD
intra_df = pd.read_csv("./dataset/drug_lab_base_order/intraHD2016_20_fix_2024-09-25.csv")
intra_df['洗腎紀錄時間']= pd.to_datetime(intra_df['洗腎紀錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間']
intra_df['中記錄時間']= pd.to_datetime(intra_df['中記錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間去時分'].dt.date
intra_df[intra_df['急診門診住院']=='2.門診  ']
data_df['洗腎紀錄時間去時分']= pd.to_datetime(data_df['洗腎紀錄時間去時分']).dt.date

In [95]:
# 修正outlier 開始/結束脈搏
# 30 <= Pulse <= 150
# 第一階段開始脈搏或結束脈搏補值：從IntraHD取第一筆或最後一筆脈搏補值
print(f"開始脈搏S: 異常共{data_df[(data_df['開始脈搏']==-9999)|(data_df['開始脈搏']==19999)].shape[0]}筆")
print(f"結束脈搏S: 異常共{data_df[(data_df['結束脈搏']==-9999)|(data_df['結束脈搏']==19999)].shape[0]}筆")
error_pulse = data_df[((data_df['開始脈搏']==-9999)|(data_df['結束脈搏']==-9999)|(data_df['開始脈搏']==19999)|(data_df['結束脈搏']==19999))][['ID','洗腎紀錄時間','洗腎紀錄時間去時分','開始脈搏','結束脈搏']]
for index, single_row in tqdm(error_pulse.iterrows()):
    if single_row['開始脈搏'] in [-9999, 19999]:
        match_intra_df = intra_df[(intra_df['ID']==single_row['ID'])&(intra_df['洗腎紀錄時間去時分']==single_row['洗腎紀錄時間去時分'])]
        if match_intra_df.shape[0]!=0:
            match_intra_df = match_intra_df.sort_values('中記錄時間')
            intra_first_bp = match_intra_df.loc[match_intra_df.index[0]]['脈搏']
            try:
                intra_first_bp = int(intra_first_bp)
            except:
                intra_first_bp = -9999
            if 30 <= intra_first_bp <= 150:
                error_pulse.loc[index,'開始脈搏'] = intra_first_bp
            else:
                error_pulse.loc[index,'開始脈搏'] = 19999.0
        else:
            error_pulse.loc[index,'開始脈搏'] = -9999.0
            
    if single_row['結束脈搏'] in [-9999, 19999]:
        match_intra_df = intra_df[(intra_df['ID']==single_row['ID'])&(intra_df['洗腎紀錄時間去時分']==single_row['洗腎紀錄時間去時分'])]
        if match_intra_df.shape[0]!=0:
            match_intra_df = match_intra_df.sort_values('中記錄時間')
            intra_first_bp = match_intra_df.loc[match_intra_df.index[-1]]['脈搏']
            try:
                intra_first_bp = int(intra_first_bp)
            except:
                intra_first_bp = -9999
            if 30 <= intra_first_bp <= 150:
                error_pulse.loc[index,'結束脈搏'] = intra_first_bp
            else:
                error_pulse.loc[index,'結束脈搏'] = 19999.0
        else:
            error_pulse.loc[index,'結束脈搏'] = -9999.0
            
print(f"開始脈搏E: 異常共{error_pulse[(error_pulse['開始脈搏']==-9999)].shape[0]}筆")
print(f"結束脈搏E: 異常共{error_pulse[(error_pulse['結束脈搏']==-9999)].shape[0]}筆")
data_df.update(error_pulse)

開始脈搏S: 異常共4197筆
結束脈搏S: 異常共4309筆


0it [00:00, ?it/s]

開始脈搏E: 異常共509筆
結束脈搏E: 異常共736筆


In [96]:
# 血壓脈搏outlier處理
pulse_check = lambda x: x if (30 <= x <= 150) and x!=-9999 else 19999
sbp_check = lambda x: x if (60 <= x <= 240) and x!=-9999 else 19999
dbp_check = lambda x: x if (30 <= x <= 140) and x!=-9999 else 19999
temp_check = lambda x: x if (32 <= x <= 40) and x!=-9999 else 19999
check_columns = ['開始血壓SBP', '開始血壓DBP','開始脈搏', '體溫', ]
data_df['開始血壓SBP'] = data_df['開始血壓SBP'].apply(sbp_check)
data_df['開始血壓DBP'] = data_df['開始血壓DBP'].apply(dbp_check)
data_df['開始脈搏'] = data_df['開始脈搏'].apply(pulse_check)
data_df['體溫'] = data_df['體溫'].apply(pulse_check)

In [97]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Raw Index,309556.0,154777.500000,89361.264302,0.000,77388.750000,154777.50000,232166.250000,309555.00
index,309556.0,179308.629007,103597.829404,4.000,90895.750000,180416.50000,268817.250000,358822.00
入院方式,309556.0,-75.957387,870.305905,-9999.000,0.000000,0.00000,0.000000,3.00
FISTULA,309556.0,0.840026,0.366583,0.000,1.000000,1.00000,1.000000,1.00
GRAFT,309556.0,0.130632,0.336998,0.000,0.000000,0.00000,0.000000,1.00
DLC,309556.0,0.000853,0.029191,0.000,0.000000,0.00000,0.000000,1.00
PERMCATH,309556.0,0.066660,0.249433,0.000,0.000000,0.00000,0.000000,1.00
開始血壓SBP,309556.0,194.661248,1015.360304,60.000,124.000000,142.00000,160.000000,19999.00
開始血壓DBP,309556.0,121.908816,977.725690,30.000,64.000000,73.00000,83.000000,19999.00
開始脈搏,309556.0,155.850877,1232.739883,30.000,71.000000,79.00000,87.000000,19999.00


In [98]:
if "第二階段處理結果-生理數據[4-脈搏補值處理].csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第二階段處理結果-生理數據[4-脈搏補值處理].csv")
else:
    # intra_df[(intra_df['ID']=='134811545')&(intra_df['洗腎紀錄時間']>'2020-8')&(intra_df['洗腎紀錄時間']<'2020-11')]
    # error_pulse.to_csv("./dataset/outlier/第二階段處理結果-脈搏補值處理outlier_error.csv", index=False, encoding='utf-8-sig')
    data_df.to_csv("./dataset/第二階段處理結果-生理數據[4-脈搏補值處理].csv", index=False, encoding='utf-8-sig')

In [99]:
# loading intraHD
intra_df = pd.read_csv("./dataset/drug_lab_base_order/intraHD2016_20_fix_2024-09-25.csv")
intra_df['洗腎紀錄時間']= pd.to_datetime(intra_df['洗腎紀錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間']
intra_df['中記錄時間']= pd.to_datetime(intra_df['中記錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間去時分'].dt.date
intra_df[intra_df['急診門診住院']=='2.門診  ']

data_df['洗腎紀錄時間去時分']= pd.to_datetime(data_df['洗腎紀錄時間去時分']).dt.date

In [100]:
# 處理intraHD脈搏
intra_df['脈搏'] = intra_df['脈搏'].astype(str)
intra_df['脈搏'] = intra_df['脈搏'].apply(lambda x: re.sub(r'[^0-9\.-]', '', x))
# 將有範圍且符合脈搏範圍的資料做平均補值, e.g: 83-106 => 94
for index, bp in (intra_df[['ID','脈搏']].iterrows()):
    if str(bp['脈搏']).count('-') >0:
        bp_split = str(bp['脈搏']).split('-')
        if bp_split[0]!='' and bp_split[-1]!='':
            if (30 <= int(bp_split[0]) <= 150) and (30 <= int(bp_split[-1]) <= 150):
                # print(bp_split,'-1',index)
                intra_df.loc[index,'脈搏'] = (int(bp_split[0])+int(bp_split[-1]))//2
            elif (30 <= int(bp_split[0])) and (30 <= int(bp_split[-1])):
                # print(bp_split,'-2',index)
                intra_df.loc[index,'脈搏'] = min([int(bp_split[0]), int(bp_split[-1])])
            else:
                # print(bp_split,'-3',index)
                intra_df.loc[index,'脈搏'] =-9999

In [101]:
intra_df['脈搏'] = pd.to_numeric(intra_df['脈搏'], errors='coerce')
intra_df['脈搏'] = intra_df['脈搏'].apply(lambda x: 19999.0 if x<30  and x!=-9999 else x)
intra_df['脈搏'] = intra_df['脈搏'].apply(lambda x: 19999.0 if x>150  and x!=-9999 else x)
intra_df['脈搏'].replace(np.nan, -9999, inplace=True)
print(intra_df.shape)
# intra_df = intra_df[intra_df['脈搏']!=9999] #過濾異常資料
print(intra_df.shape)

(1941900, 22)
(1941900, 22)


In [102]:
# # 第二階段開始脈搏或結束脈搏檢測：從IntraHD取第一筆或最後一筆脈搏補值
# # 規則："PH開始-IH第一筆" 或是 "PH開始-IH第二筆"的差>30，則代表任一次脈搏紀錄有問題
# # 排除intra脈搏一開始就記錯的rows：異常標記9999 or nan
# # e.g: P:130, I1:90, I2:120，嘗試取IntraHD-1: 90為主要開始脈搏
# # 以60-100之間的為主，如果不在區間內取最接近60-100的脈搏
# # 開始脈搏
# filter_data_df = data_df[data_df['開始脈搏']!=9999]#排除開始脈搏已經為異常的數值
# filter_data_df = filter_data_df[['Raw Index', 'ID','洗腎紀錄時間去時分','開始脈搏','結束脈搏']]
# filter_intra_df = intra_df[['ID','洗腎紀錄時間去時分','中記錄時間','脈搏']]
# merged_df = pd.merge(filter_data_df, filter_intra_df, on=['ID', '洗腎紀錄時間去時分'], how='inner')
# merged_df = merged_df.sort_values(by=['ID','洗腎紀錄時間去時分','中記錄時間',], ascending=True)
# merged_df['脈搏'] = pd.to_numeric(merged_df['脈搏'], errors='coerce')
# merged_df['開始脈搏差'] = abs(merged_df['開始脈搏']-merged_df['脈搏'])
# merged_df['開始脈搏差'].replace(np.nan, 9999, inplace=True)
# grouped = merged_df.groupby(['ID', '洗腎紀錄時間去時分'])

In [103]:
# cant_adjust_group_S = []
# pulse_adjust_index_S = []
# avg = (100+60)//2
# for group_name, group_data in tqdm(grouped):
#     if group_data.shape[0]>=2:
#         first_pulse_diff = group_data.loc[group_data.index[0]]['開始脈搏差']
#         second_pulse_diff = group_data.loc[group_data.index[1]]['開始脈搏差']
#         if (first_pulse_diff>30 or second_pulse_diff>30):
#             start_pulse = group_data.loc[group_data.index[0],'開始脈搏']
#             first_pulse = group_data.loc[group_data.index[0],'脈搏']
#             second_pulse = group_data.loc[group_data.index[1],'脈搏']
#             if 60<=start_pulse<=100:
#                 data_df.loc[group_data['Raw Index'].iloc[0], '開始脈搏'] = start_pulse
                
#                 # pulse_adjust_index_S.append(group_data.iloc[0]['Raw Index'])
#             elif 60<=first_pulse<=100:
#                 data_df.loc[group_data['Raw Index'].iloc[0], '開始脈搏'] = first_pulse
                
#                 pulse_adjust_index_S.append(group_data.iloc[0]['Raw Index'])
#             elif 60<=second_pulse<=100:
#                 data_df.loc[group_data['Raw Index'].iloc[0], '開始脈搏'] = second_pulse
                
#                 pulse_adjust_index_S.append(group_data.iloc[0]['Raw Index'])
#             else:
#                 pluse_list = [start_pulse, first_pulse, second_pulse]
#                 min_pluse = pluse_list[np.argmin([abs(avg-start_pulse),abs(avg-first_pulse),abs(avg-second_pulse)])]
#                 if min_pluse>9000:
#                     data_df.loc[group_data['Raw Index'].iloc[0], '開始脈搏'] = start_pulse
#                 else:
#                     data_df.loc[group_data['Raw Index'].iloc[0], '開始脈搏'] = min_pluse
#                     pulse_adjust_index_S.append(group_data.iloc[0]['Raw Index'])       
#     else:
#         cant_adjust_group_S.append(group_name)
#     # break
# print(f"共調整{len(pulse_adjust_index_S)}，無法調整資料共{len(cant_adjust_group_S)}")

In [104]:
# # 第二階段開始脈搏或結束脈搏檢測：從IntraHD取第一筆或最後一筆脈搏補值
# # 規則："PH開始-IH第一筆" 或是 "PH開始-IH第二筆"的差>30，則代表任一次脈搏紀錄有問題
# # 排除intra脈搏一開始就記錯的rows：異常標記9999 or nan
# # e.g: P:130, I1:90, I2:120，嘗試取IntraHD-1: 90為主要開始脈搏
# # 以60-100之間的為主，如果不在區間內取最接近60-100的脈搏
# # 結束脈搏
# filter_data_df = data_df[data_df['結束脈搏']!=9999]#排除開始脈搏已經為異常的數值
# filter_data_df = filter_data_df[['Raw Index', 'ID','洗腎紀錄時間去時分','開始脈搏','結束脈搏']]
# filter_intra_df = intra_df[['ID','洗腎紀錄時間去時分','中記錄時間','脈搏']]
# merged_df = pd.merge(filter_data_df, filter_intra_df, on=['ID', '洗腎紀錄時間去時分'], how='inner')
# merged_df = merged_df.sort_values(by=['ID','洗腎紀錄時間去時分','中記錄時間',], ascending=True)
# merged_df['脈搏'] = pd.to_numeric(merged_df['脈搏'], errors='coerce')
# merged_df['結束脈搏差'] = abs(merged_df['結束脈搏']-merged_df['脈搏'])
# merged_df['結束脈搏差'].replace(np.nan, 9999, inplace=True)
# grouped = merged_df.groupby(['ID', '洗腎紀錄時間去時分'])

In [105]:
# cant_adjust_group_E = []
# pulse_adjust_index_E = []
# avg = (100+60)//2
# for group_name, group_data in tqdm(grouped):
#     if group_data.shape[0]>=2:
#         first_pulse_diff = group_data.loc[group_data.index[-1]]['結束脈搏差']
#         second_pulse_diff = group_data.loc[group_data.index[-2]]['結束脈搏差']
#         if (first_pulse_diff>30 or second_pulse_diff>30):
#             end_pulse = group_data.loc[group_data.index[0],'結束脈搏']
#             first_pulse = group_data.loc[group_data.index[-1],'脈搏']
#             second_pulse = group_data.loc[group_data.index[-2],'脈搏']
#             if 60<=end_pulse<=100:
#                 data_df.loc[group_data['Raw Index'].iloc[0], '結束脈搏'] = end_pulse
                
#                 # pulse_adjust_index_E.append(group_data.iloc[0]['Raw Index'])
#             elif 60<=first_pulse<=100:
#                 data_df.loc[group_data['Raw Index'].iloc[0], '結束脈搏'] = first_pulse
                
#                 pulse_adjust_index_E.append(group_data.iloc[0]['Raw Index'])
#             elif 60<=second_pulse<=100:
#                 data_df.loc[group_data['Raw Index'].iloc[0], '結束脈搏'] = second_pulse
                
#                 pulse_adjust_index_E.append(group_data.iloc[0]['Raw Index'])
#             else:
#                 pluse_list = [end_pulse, first_pulse, second_pulse]
#                 min_pluse = pluse_list[np.argmin([abs(avg-end_pulse),abs(avg-first_pulse),abs(avg-second_pulse)])]
#                 if min_pluse>9000:
#                     data_df.loc[group_data['Raw Index'].iloc[0], '結束脈搏'] = end_pulse
#                 else:
#                     data_df.loc[group_data['Raw Index'].iloc[0], '結束脈搏'] = min_pluse
#                     pulse_adjust_index_E.append(group_data.iloc[0]['Raw Index'])       
#     else:
#         cant_adjust_group_E.append(group_name)
#     # break
# print(f"共調整{len(pulse_adjust_index_E)}，無法調整資料共{len(cant_adjust_group_E)}")

In [106]:
intra_df.head(2)

,ID,洗腎紀錄時間,急診門診住院,中記錄時間,記錄者,血壓,脈搏,呼吸,血液流速,靜脈壓,TMPDP,超過濾率,溫度,傳導濃度,管路固定,生理鹽水,肝素使用量,處置其他,症狀處置,洗腎紀錄時間去時分,sbp,dbp
0,00E08Z252,2016-01-16 12:38:00,2.門診,2016-01-16 13:52:00,87052,125/43,70.0,18,260,158,36/121,0.29,36.1,140,Yes,NaN,NaN,NaN,NaN,2016-01-16,125.0,43.0
1,00E08Z252,2016-01-16 12:38:00,2.門診,2016-01-16 14:13:00,87052,-9999/-9999,-9999.0,18,260,157,49/109,0.29,35.2,140,Yes,NaN,NaN,NaN,NaN,2016-01-16,-9999.0,-9999.0


In [107]:
if "第二階段處理結果-生理數據[5-生理特徵處理完畢].csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第二階段處理結果-生理數據[5-生理特徵處理完畢].csv")
else:
    data_df.to_csv("./dataset/第二階段處理結果-生理數據[5-生理特徵處理完畢].csv", index=False, encoding='utf-8-sig')

## 修正不合理範圍區間的檢驗特徵
### #######所有lab檢驗值都不應該有0存在，除了類別變數(max:1, min:0)#######
### 將0的數據，修正為9999視為異常資料

In [108]:
# 修正不合理範圍區間的檢驗特徵
# 1. 如果生理數值為0或小於0則設為9999
# 2. Intact PTH在先前已改成非類別，該階段略過此數值

from utils.feature_check_miss import *
non_check_cols = ['Raw Index', 'ID', '洗腎紀錄時間', '洗腎紀錄時間去時分', '急診門診住院', '處置其他結束', '開始血壓SBP', '開始血壓DBP', '開始脈搏', '體溫', '結束脈搏',
                  'Hollow fiber clot', '生日', '性別', '算年齡', 'age', '體重1開始', '體重2標準體重', '體重結束', '體重實際脫水', '體重機器顯示', '體重3應脫水', '分母體重', 
                  '每公斤脫水量(ml/kg)', 'Intact PTH']
cat_cols = ['FISTULA', 'GRAFT', 'DLC', 'PERMCATH', 'Catheter','HBsAg','HBc','HBs','HCV','HBV','有無糖尿病']
non_check_cols = non_check_cols + cat_cols
filter_cols = []
for cols in data_df.columns[5::]:
    if cols not in non_check_cols:
        # print(f"異常檢測 - {cols}")
        filter_cols.append(cols)
        temp_ = data_df[(data_df[cols]!=19999)&(data_df[cols]!=-9999)][cols]
        error_ = data_df[(data_df[cols]==19999)&(data_df[cols]==-9999)][cols]
        if cols in Feature_dict.keys():
            if Feature_dict[cols]['Condition']!=None:
                failed_ = temp_.apply(Feature_dict[cols]['Condition'])
                # 將不合理的index提取後全部替補為9999.0
            failed_ = temp_.apply(lambda x: False if (x in [0, 0.0 ,'0']) else True)
            failed_index = failed_[failed_==False].index
            data_df.loc[failed_index, cols] =19999.0
            if len(failed_)-failed_.sum()!=0:
                print(f"異常檢測 - {cols}")
                print(f"不合理數值數量:\t{len(failed_)-failed_.sum()}")
                print(f"Min:{float(min(temp_)):.3f} \t Max:{float(max(temp_)):.3f} \t 異常計數:{len(error_)}")
                print("============================================")
                print("\n") 

異常檢測 - BUN
不合理數值數量:	6
Min:0.000 	 Max:242.000 	 異常計數:0


異常檢測 - CRP
不合理數值數量:	160
Min:0.000 	 Max:42.660 	 異常計數:0




In [109]:
if "第二階段處理結果-生理數據[6-新特徵處理完畢].csv" in os.listdir("./dataset/"):
    data_df = pd.read_csv("./dataset/第二階段處理結果-生理數據[6-新特徵處理完畢].csv")
else:
    data_df.to_csv("./dataset/第二階段處理結果-生理數據[6-新特徵處理完畢].csv", index=False, encoding='utf-8-sig')

In [110]:
print(len(set(data_df['ID'])))

1051


# 處理IntraHD、PeriHD

In [111]:
# 新特徵部分皆已數值方面做處理
# 列出所有新舊特徵的list
# K: 2.0-10.0 (3.5-5.3)
# Glucose AC和Glucose PC以AC為主，PC移除
data_df = data_df.drop(['Glucose PC'], axis=1, errors='ignore')
# 丟棄LDL Cholesterol，原因：遺失值過多
data_df = data_df.drop(['LDL Cholesterol'], axis=1, errors='ignore')
# 丟棄Hollow fiber clot，原因：較難用於分析
data_df = data_df.drop(['Hollow fiber clot'], axis=1, errors='ignore')
print(f"過濾前資料量：{data_df.shape[0]}")
print(len(set(data_df['ID'])))

過濾前資料量：309556
1051


# 年齡過濾，只挑選大於等於20歲以上的數據

In [112]:
# # 處理入院方式 => 'nan', '助行器', '推床', '步行', '輪椅'
# str_cat = {'步行':0,'助行器':1, '輪椅':2, '推床':3, 'nan':4}
# data_df['入院方式'] = data_df['入院方式'].astype(str).apply(lambda x: str_cat[x])

In [113]:
data_df = data_df[data_df['age']>=20]
print(data_df.shape)
data_df.shape[0]

(308926, 65)


308926

In [129]:
data_df['體重實際脫水'] = data_df['體重實際脫水'].apply(lambda x: 19999 if x>100 else x)

In [142]:
filter = ['index','Raw Index','ID','洗腎紀錄時間','洗腎紀錄時間去時分','急診門診住院','處置其他結束','生日','算年齡','體重3應脫水']
for cols in data_df.columns:
    if cols in filter:
        pass
    else:
        filter_df_error = data_df[data_df[cols]==19999]
        filter_df_miss = data_df[data_df[cols]==-9999]
        print("=============[ {} ]=================".format(cols))
        print("Max:{} |  Min:{}".format(data_df[cols].max(), data_df[cols].min()))
        print("MissR: {}% | ErrR:{}%".format((len(filter_df_miss)/data_df.shape[0])*100, (len(filter_df_error)/data_df.shape[0])*100))
        print("==============================")
        
    # break

=============[ 入院方式 ]=================
Max:3 |  Min:-9999
MissR: 0.7623184840382486% | ErrR:0.0%
=============[ FISTULA ]=================
Max:1.0 |  Min:0.0
MissR: 0.0% | ErrR:0.0%
=============[ GRAFT ]=================
Max:1.0 |  Min:0.0
MissR: 0.0% | ErrR:0.0%
=============[ DLC ]=================
Max:1.0 |  Min:0.0
MissR: 0.0% | ErrR:0.0%
=============[ PERMCATH ]=================
Max:1.0 |  Min:0.0
MissR: 0.0% | ErrR:0.0%
=============[ 開始血壓SBP ]=================
Max:19999.0 |  Min:60.0
MissR: 0.0% | ErrR:0.26252241637155826%
=============[ 開始血壓DBP ]=================
Max:19999.0 |  Min:30.0
MissR: 0.0% | ErrR:0.24180548092423426%
=============[ 開始脈搏 ]=================
Max:19999.0 |  Min:30.0
MissR: 0.0% | ErrR:0.3852055184736798%
=============[ 體溫 ]=================
Max:19999.0 |  Min:30.6
MissR: 0.0% | ErrR:1.3310631024905641%
=============[ 結束脈搏 ]=================
Max:19999.0 |  Min:-9999.0
MissR: 0.23824475764422548% | ErrR:0.07768850792746483%
=============[ Catheter ]=======

In [113]:
data_df = data_df[data_df['開始血壓SBP']!=-9999]
data_df = data_df[data_df['開始血壓DBP']!=-9999]
print(f"血壓過濾後資料量：{data_df.shape[0]}")
data_df = data_df[data_df['體重實際脫水']!=-9999]
print(f"實際脫水量過濾後資料量：{data_df.shape[0]}")
print(len(set(data_df['ID'])))

血壓過濾後資料量：309556
實際脫水量過濾後資料量：309556
1051


In [ ]:
# loading intraHD
intra_df = pd.read_csv("./dataset/drug_lab_base_order/intraHD2016_20_fix_2024-09-25.csv")
intra_df['洗腎紀錄時間']= pd.to_datetime(intra_df['洗腎紀錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間']
intra_df['中記錄時間']= pd.to_datetime(intra_df['中記錄時間'])
intra_df['洗腎紀錄時間去時分']= intra_df['洗腎紀錄時間去時分'].dt.date
intra_df[intra_df['急診門診住院']=='2.門診  ']
print("IntraHD ID數量：",len(set(intra_df['ID'])))
print("PeriHD  ID數量：",len(set(data_df['ID'])))
data_df['洗腎紀錄時間去時分']= pd.to_datetime(data_df['洗腎紀錄時間去時分']).dt.date

In [ ]:
# 過濾PeriHD: 去除少於12筆記錄的ID
# grouped = data_df.groupby(['ID'])
# print(f"篩選前ID數量：{len(data_df['ID'].drop_duplicates())}")
# drop_ID = []
# for group_name, group_data in (grouped):
#     if group_data.shape[0]<12:
#         drop_ID.append(group_data['ID'].iloc[0])
        
# data_df = data_df[~data_df['ID'].isin(drop_ID)]
# print(f"篩選後ID數量：{len(data_df['ID'].drop_duplicates())}")

In [ ]:
if "第二階段處理結果-生理數據[99-Finished].csv" in os.listdir("./dataset/DATA_B/"):
    data_df = pd.read_csv("./dataset/DATA_B/第二階段處理結果-生理數據[99-Finished].csv")
else:
    data_df.to_csv("./dataset/DATA_B/第二階段處理結果-生理數據[99-Finished].csv", index=False, encoding='utf-8-sig')

# 數據檢查

In [ ]:
# 過濾IntraHD：保留PeriHD有的ID
print(f"篩選前ID數量：{len(intra_df['ID'].drop_duplicates())}")
print(f"過濾前資料量：{intra_df.shape[0]}")
intra_df = intra_df[intra_df['ID'].isin(data_df['ID'].drop_duplicates())]
print(f"篩選後ID數量：{len(intra_df['ID'].drop_duplicates())}")
print(f"過濾後資料量：{intra_df.shape[0]}")
# 篩選前ID數量：3995
# 過濾前資料量：1941900
# 篩選後ID數量：771
# 過濾後資料量：1221345

## 由於前半段略過很多刪除和過濾的程序，該階段需要做最後一次重複資料的檢查

In [ ]:
print(data_df.shape)
data_df = data_df.drop_duplicates(subset=['index'])
data_df.reset_index(inplace=True, drop=True)
print(data_df.shape)

In [ ]:
# 以PeriHD作為主表串連IntraHD，每一筆PeriHD都必須有IntraHD，只要沒有紀錄則移除(Inner)。
# 確保滿足NLP自動標記時須有兩組資料的條件限制

In [ ]:
flatten_intra = {
    "ID":[],
    "洗腎紀錄時間去時分":[],
    "處置其他":[],
    "症狀處置":[],
    "sbp":[],
    "dbp":[],
}
intra_grouped = intra_df.groupby(['ID','洗腎紀錄時間去時分'])
for group_name, group_data in tqdm(intra_grouped):
    flatten_intra["ID"].append(group_data['ID'].iloc[0])
    flatten_intra["洗腎紀錄時間去時分"].append(str(group_data['洗腎紀錄時間去時分'].iloc[0]))
    flatten_intra["處置其他"].append(list(group_data['處置其他'].values))
    flatten_intra["症狀處置"].append(list(group_data['症狀處置'].values))
    flatten_intra["sbp"].append(list(group_data['sbp'].values))
    flatten_intra["dbp"].append(list(group_data['dbp'].values))
    # break
flatten_intra = pd.DataFrame(flatten_intra)
flatten_intra.to_pickle("./dataset/DATA_B/第一階段處理結果-FlattenIntraHD.pkl")
PeriHD_merge = pd.merge(data_df, flatten_intra, on=['ID', '洗腎紀錄時間去時分'], how='inner')
PeriHD_merge.to_pickle("./dataset/DATA_B/第一階段處理結果-FinishedPeriHD.pkl")

In [ ]:
PeriHD_merge.shape